In [1]:
!nvidia-smi

Thu Jul 16 08:23:44 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [54]:
!pip install transformers


import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer , BertModel
from torch.utils.data import TensorDataset, random_split



if torch.cuda.is_available():        
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


df = pd.read_csv("/content/only_captions.csv", header=None)
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
sentences = df[1].values
sentences = np.delete(sentences, 0)


print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
input_ids = []
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(sent, add_special_tokens = True, max_length = 18, pad_to_max_length = True, return_attention_mask = True,   
                        return_tensors = 'pt', truncation=True)
    input_ids.append(encoded_dict['input_ids'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
segments_ids = []
for i in range(input_ids.size()[0]):
  segments_id = [1] * input_ids.size()[1]
  segments_ids.append(segments_id)
segments_ids = torch.tensor(segments_ids)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])


There are 1 GPU(s) available.
We will use the GPU: Tesla T4
Number of training sentences: 88,551

Loading BERT tokenizer...
Original:  a bird with a very long wing span and a long pointed beak.
Token IDs: tensor([  101,  1037,  4743,  2007,  1037,  2200,  2146,  3358,  8487,  1998,
         1037,  2146,  4197, 23525,  1012,   102,     0,     0])


In [ ]:
import os
os.chdir('/content')
!rm -r sample_data

#clone repo AttnGAN
!git clone https://github.com/taoxugit/AttnGAN.git

#Changing Working dirctory to data
os.chdir('/content/AttnGAN/data/')
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1O_LtUP9sch09QH3s_EBAgLEctBQ5JBSJ' -O birds.zip
!unzip -q birds.zip
!rm birds.zip
!rm -r __MACOSX/

#Changing Working dirctory to birds
os.chdir('/content/AttnGAN/data/birds/')
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45" -O CUB_200_2011.tgz && rm -rf /tmp/cookies.txt
!tar zxf  CUB_200_2011.tgz
!rm CUB_200_2011.tgz

#Changing Working dirctory to code
os.chdir('/content/AttnGAN/code/')
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Wr3lQajG7m6Bi3rYFTJb6mwE_d8su111' -O Pillow.rar
!unrar x  Pillow.rar
!rm Pillow.rar

In [ ]:
import os
os.chdir('/content/AttnGAN/code/')

import os.path as osp
import sys
import time
import random
import pprint
import datetime
import dateutil.tz
import numpy as np
import numpy.random as random
import pandas as pd
import pickle
from easydict import EasyDict as edict

from PIL import Image, ImageDraw, ImageFont
from copy import deepcopy
import skimage.transform

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from torchvision import models
import torchvision.transforms as transforms

import torch.utils.model_zoo as model_zoo
import torch.utils.data as data

from nltk.tokenize import RegexpTokenizer
from collections import defaultdict


__C = edict()
cfg = __C
__C.DATASET_NAME = 'birds'
__C.CONFIG_NAME = 'DAMSM'
__C.DATA_DIR = '../data/birds'
__C.GPU_ID = 0
__C.CUDA = True
__C.WORKERS = 1
__C.B_VALIDATION = False

__C.TREE = edict()
__C.TREE.BRANCH_NUM = 1
__C.TREE.BASE_SIZE = 299

# Training options
__C.TRAIN = edict()
__C.TRAIN.BATCH_SIZE = 48
__C.TRAIN.MAX_EPOCH = 600
__C.TRAIN.SNAPSHOT_INTERVAL = 50
__C.TRAIN.DISCRIMINATOR_LR = 0.0002
__C.TRAIN.GENERATOR_LR = 0.0002
__C.TRAIN.ENCODER_LR = 0.002
__C.TRAIN.RNN_GRAD_CLIP = 0.25
__C.TRAIN.FLAG = True
__C.TRAIN.NET_E = ''
__C.TRAIN.NET_G = ''
__C.TRAIN.B_NET_D = True
__C.TRAIN.SMOOTH = edict()
__C.TRAIN.SMOOTH.GAMMA1 = 4.0
__C.TRAIN.SMOOTH.GAMMA3 = 10.0
__C.TRAIN.SMOOTH.GAMMA2 = 5.0
__C.TRAIN.SMOOTH.LAMBDA = 1.0

# Modal options
__C.GAN = edict()
__C.GAN.DF_DIM = 64
__C.GAN.GF_DIM = 128
__C.GAN.Z_DIM = 100
__C.GAN.CONDITION_DIM = 100
__C.GAN.R_NUM = 2
__C.GAN.B_ATTENTION = True
__C.GAN.B_DCGAN = False

__C.TEXT = edict()
__C.TEXT.CAPTIONS_PER_IMAGE = 10
__C.TEXT.EMBEDDING_DIM = 256
__C.TEXT.WORDS_NUM = 18



def get_imgs(img_path, imsize, bbox=None, transform=None, normalize=None):
    img = Image.open(img_path).convert('RGB')
    width, height = img.size
    if bbox is not None:
        r = int(np.maximum(bbox[2], bbox[3]) * 0.75)
        center_x = int((2 * bbox[0] + bbox[2]) / 2)
        center_y = int((2 * bbox[1] + bbox[3]) / 2)
        y1 = np.maximum(0, center_y - r)
        y2 = np.minimum(height, center_y + r)
        x1 = np.maximum(0, center_x - r)
        x2 = np.minimum(width, center_x + r)
        img = img.crop([x1, y1, x2, y2])

    if transform is not None:
        img = transform(img)

    ret = []
    if cfg.GAN.B_DCGAN:
        ret = [normalize(img)]
    else:
        for i in range(cfg.TREE.BRANCH_NUM):
            # print(imsize[i])
            if i < (cfg.TREE.BRANCH_NUM - 1):
                re_img = transforms.Scale(imsize[i])(img)
            else:
                re_img = img
            ret.append(normalize(re_img))

    return ret

def prepare_data(data):
    imgs, captions, captions_lens, class_ids, keys, input_ids, segments_ids = data

    # sort data by the length in a decreasing order !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!MARKER!!!!!!!!!!!!!!!!!!!!!!!!
    sorted_cap_lens, sorted_cap_indices = torch.sort(captions_lens, 0, True)

    real_imgs = []
    for i in range(len(imgs)):
        imgs[i] = imgs[i][sorted_cap_indices]
        if cfg.CUDA:
            real_imgs.append(Variable(imgs[i]).cuda())
        else:
            real_imgs.append(Variable(imgs[i]))

    captions = captions[sorted_cap_indices].squeeze()
    class_ids = class_ids[sorted_cap_indices].numpy()
    # sent_indices = sent_indices[sorted_cap_indices]
    keys = [keys[i] for i in sorted_cap_indices.numpy()]
    # print('keys', type(keys), keys[-1])  # list
    if cfg.CUDA:
        captions = Variable(captions).cuda()
        sorted_cap_lens = Variable(sorted_cap_lens).cuda()
    else:
        captions = Variable(captions)
        sorted_cap_lens = Variable(sorted_cap_lens)

    input_ids = input_ids.cuda()
    segments_ids = segments_ids.cuda()

    return [real_imgs, captions, sorted_cap_lens,
            class_ids, keys, input_ids, segments_ids]

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

def build_super_images(real_imgs, captions, ixtoword,
                        attn_maps, att_sze, lr_imgs=None,
                        batch_size=cfg.TRAIN.BATCH_SIZE,
                        max_word_num=cfg.TEXT.WORDS_NUM):
    
    
    COLOR_DIC = {0:[128,64,128],  1:[244, 35,232],
                2:[70, 70, 70],  3:[102,102,156],
                4:[190,153,153], 5:[153,153,153],
                6:[250,170, 30], 7:[220, 220, 0],
                8:[107,142, 35], 9:[152,251,152],
                10:[70,130,180], 11:[220,20, 60],
                12:[255, 0, 0],  13:[0, 0, 142],
                14:[119,11, 32], 15:[0, 60,100],
                16:[0, 80, 100], 17:[0, 0, 230],
                18:[0,  0, 70],  19:[0, 0,  0]}
    FONT_MAX = 50

    
    build_super_images_start_time = time.time()
    nvis = 8
    real_imgs = real_imgs[:nvis]
    if lr_imgs is not None:
        lr_imgs = lr_imgs[:nvis]
    if att_sze == 17:
        vis_size = att_sze * 16
    else:
        vis_size = real_imgs.size(2)

    text_convas = \
        np.ones([batch_size * FONT_MAX,
                 (max_word_num + 2) * (vis_size + 2), 3],
                dtype=np.uint8)


    # print("keyword |||||||||||||||||||||||||||||||")
    # print("max_word_num : " , max_word_num)
    # print("keyword |||||||||||||||||||||||||||||||")
    for i in range(max_word_num):
        istart = (i + 2) * (vis_size + 2)
        iend = (i + 3) * (vis_size + 2)
        text_convas[:, istart:iend, :] = COLOR_DIC[i]


    real_imgs = \
        nn.Upsample(size=(vis_size, vis_size), mode='bilinear')(real_imgs)
    # [-1, 1] --> [0, 1]
    real_imgs.add_(1).div_(2).mul_(255)
    real_imgs = real_imgs.data.numpy()
    # b x c x h x w --> b x h x w x c
    real_imgs = np.transpose(real_imgs, (0, 2, 3, 1))
    pad_sze = real_imgs.shape
    middle_pad = np.zeros([pad_sze[2], 2, 3])
    post_pad = np.zeros([pad_sze[1], pad_sze[2], 3])
    if lr_imgs is not None:
        lr_imgs = \
            nn.Upsample(size=(vis_size, vis_size), mode='bilinear')(lr_imgs)
        # [-1, 1] --> [0, 1]
        lr_imgs.add_(1).div_(2).mul_(255)
        lr_imgs = lr_imgs.data.numpy()
        # b x c x h x w --> b x h x w x c
        lr_imgs = np.transpose(lr_imgs, (0, 2, 3, 1))

    # batch x seq_len x 17 x 17 --> batch x 1 x 17 x 17
    seq_len = max_word_num
    img_set = []
    num = nvis  # len(attn_maps)

    text_map, sentences = \
        drawCaption(text_convas, captions, ixtoword, vis_size)
    text_map = np.asarray(text_map).astype(np.uint8)

    bUpdate = 1
    for i in range(num):
        #print ("loop " , i ," of " , num == 8)
        attn = attn_maps[i].cpu().view(1, -1, att_sze, att_sze)
        # --> 1 x 1 x 17 x 17
        attn_max = attn.max(dim=1, keepdim=True)
        attn = torch.cat([attn_max[0], attn], 1)
        #
        attn = attn.view(-1, 1, att_sze, att_sze)
        attn = attn.repeat(1, 3, 1, 1).data.numpy()
        # n x c x h x w --> n x h x w x c
        attn = np.transpose(attn, (0, 2, 3, 1))
        num_attn = attn.shape[0]
        #
        img = real_imgs[i]
        if lr_imgs is None:
            lrI = img
        else:
            lrI = lr_imgs[i]
        
        row = [lrI, middle_pad]
        #print("rowwwwwwwwwwwwwwwww : ", row)
        row_merge = [img, middle_pad]
        row_beforeNorm = []
        minVglobal, maxVglobal = 1, 0
        for j in range(num_attn):
            #print ("looop " , j , " of " , seq_len+1)
            one_map = attn[j]
            #print("First one map : " , one_map.shape)
            #print("attn.shape : " , attn.shape)

            
            # print("if (vis_size // att_sze) > 1: " ,  (vis_size // att_sze) > 1)
            # print("vis_size : " , vis_size)
            # print("att_sze : " , att_sze)
            # print("vis_size//att_sze : " , vis_size//att_sze)
            
            if (vis_size // att_sze) > 1:
                one_map = \
                    skimage.transform.pyramid_expand(one_map, sigma=20,
                                                     upscale=vis_size // att_sze)
            #    print("one_map in if : " , one_map.shape)

            
            row_beforeNorm.append(one_map)
            #print("row_beforeNorm.append(one_map)" ,len(row_beforeNorm))
            minV = one_map.min()
            maxV = one_map.max()
            if minVglobal > minV:
                minVglobal = minV
            if maxVglobal < maxV:
                maxVglobal = maxV
            #print("seq_len : " , seq_len)
        for j in range(seq_len + 1):
            #print ("loooop " , j , " of " , seq_len+1)
            
            if j < num_attn:
                one_map = row_beforeNorm[j]
                one_map = (one_map - minVglobal) / (maxVglobal - minVglobal)
                one_map *= 255
                #
                # print ("PIL_im = " , Image.fromarray(np.uint8(img)))
                # print ("PIL_att = " , Image.fromarray(np.uint8(one_map[:,:,:3])))
                # print ("img.size( :" , img.shape)
                # print ("one_map.size( :" , one_map.shape)
                PIL_im = Image.fromarray(np.uint8(img))
                PIL_att = Image.fromarray(np.uint8(one_map[:,:,:3]))
                merged = \
                    Image.new('RGBA', (vis_size, vis_size), (0, 0, 0, 0))
                #print ("merged : " , merged.size)
                mask = Image.new('L', (vis_size, vis_size), (210))
                #print (" mask  : " , mask.size)
                merged.paste(PIL_im, (0, 0))
                #print (" merged.paste(PIL_im)  : " , merged.size )
                ############################################################
                merged.paste(PIL_att, (0, 0), mask)
                #print (" merged.paste(PIL_att)  : " ,  merged.size)#########################
                merged = np.array(merged)[:, :, :3]
                #print ("  np.array(merged)[:::3] : " , merged.size )#########################
                ############################################################
            else:
                #print (" IN THE ELSE post_pad : " , post_pad.shape)
                one_map = post_pad
                #print (" one_map  : " , one_map.shape )
                merged = post_pad
                #print ("  OUTTING THE ELSE : " , merged.shape )
            
            #print ("  row : " , len(row))
            row.append(one_map[:,:,:3])
            #print ("  row.appedn(one_map) : " , len(row))
            row.append(middle_pad)
            #print ("  row.append(middle_pad) : " , len(row))
            #
            #print ("  row_merge : " , len(row_merge))
            row_merge.append(merged)
            #print ("  row_merge.append(mereged) : " , len(row_merge) )
            row_merge.append(middle_pad)
            #print ("  row_merge.append(middle_pad) : " , len(row_merge) )
        ####################################################################
        # print("row.shape : ", len(row))
        # for i in range(len(row)):
        #   print('arr', i,   
        #         " => dim0:", len(row[i]),
        #         " || dim1:", len(row[i][0]),
        #         " || dim2:", len(row[i][0][0]))
        # #print(row)
        # print("row[0].shape : ", len(row[0]))
        # #print(row[0])
        # print("row[0][0].shape : ", len(row[0][0]))
        # #print(row[0][0])
        # print("row[0][0][0].shape : ", len(row[0][0][0]))
        # #print(row[0][0][0])

        # print("row[1].shape : ", len(row[1]))
        # #print(row[1])
        # print("row[1][0].shape : ", len(row[1][0]))
        # #print(row[1][0])
        # print("row[1][0][0].shape : ", len(row[1][0][0]))
        # #print(row[1][0][0])

        # print("row[2].shape : ", len(row[2]))
        # #print(row[2])
        # print("row[2][0].shape : ", len(row[2][0]))
        # #print(row[2][0])
        # print("row[2][0][0].shape : ", len(row[2][0][0]))
        # #print(row[2][0][0])

        # print("row[3].shape : ", len(row[3]))
        # #print(row[2])
        # print("row[3][0].shape : ", len(row[3][0]))
        # #print(row[2][0])
        # print("row[3][0][0].shape : ", len(row[3][0][0]))
        # #print(row[2][0][0])

        # print("row[4].shape : ", len(row[4]))
        # #print(row[2])
        # print("row[4][0].shape : ", len(row[4][0]))
        # #print(row[2][0])
        # print("row[4][0][0].shape : ", len(row[4][0][0]))
        #print(row[2][0][0])

        

        
        
        row = np.concatenate(row, 1)
        #print (" row.conatent(1)  : " ,  len(row))########################################
        row_merge = np.concatenate(row_merge, 1)
        #print ("   : " , )############################
        ####################################################################
        txt = text_map[i * FONT_MAX: (i + 1) * FONT_MAX]
        if txt.shape[1] != row.shape[1]:
            print('txt', txt.shape, 'row', row.shape)
            bUpdate = 0
            break
        #####################################################################
        row = np.concatenate([txt, row, row_merge], 0)#######################
        img_set.append(row)##################################################
        #####################################################################
    
    # print("keyword |||||||||||||||||||||||||||||||")
    # print("bUpdate : " , bUpdate)
    # print("keyword |||||||||||||||||||||||||||||||")
    if bUpdate:
        img_set = np.concatenate(img_set, 0)
        img_set = img_set.astype(np.uint8)
        print("keyTime |||||||||||||||||||||||||||||||")
        print("build_super_images_time : " , time.time() - build_super_images_start_time)
        print("KeyTime |||||||||||||||||||||||||||||||")
        return img_set, sentences
    else:
        print("keyTime |||||||||||||||||||||||||||||||")
        print("build_super_images_start_time : " , time.time() - build_super_images_start_time)
        print("KeyTime |||||||||||||||||||||||||||||||")
        return None

def conv1x1(in_planes, out_planes, bias=False):
    "1x1 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1,
                     padding=0, bias=bias)


class TextDataset(data.Dataset):
    def __init__(self, data_dir, split='train',
                    base_size=64,
                    transform=None, target_transform=None, input_ids=None, segments_ids=None, sentences=None):
        self.transform = transform
        self.norm = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        self.target_transform = target_transform
        self.embeddings_num = cfg.TEXT.CAPTIONS_PER_IMAGE

        self.imsize = []# [299]
        for i in range(cfg.TREE.BRANCH_NUM):
            self.imsize.append(base_size)
            base_size = base_size * 2
        print("self.imsize", self.imsize)

        self.data = []
        self.data_dir = data_dir
        if data_dir.find('birds') != -1:
            self.bbox = self.load_bbox() # 11788 long dictionry with key as image name and value is 4 ints list bounding box
        else:
            self.bbox = None
        split_dir = os.path.join(data_dir, split)

        self.filenames, self.captions, self.ixtoword, self.wordtoix, self.n_words = self.load_text_data(data_dir, split)
        #filenames: List of 8855 text items of image names
        #captions: List of 8855 varible lengths captions -in range 9-18 -
        #ixtoword: dictionry  of 5450 index [key] to word [value] pairs
        #wordtoix: dictionry  of 5450 word [key] to index [value] pairs
        #n_words: 5450

        self.class_id = self.load_class_id(split_dir, len(self.filenames)) #200 classes, len:8855

        self.number_example = len(self.filenames) #8855

        self.input_ids = input_ids
        self.segments_ids = segments_ids
        self.sentences = sentences

    def load_bbox(self):
        data_dir = self.data_dir
        bbox_path = os.path.join(data_dir, 'CUB_200_2011/bounding_boxes.txt')
        df_bounding_boxes = pd.read_csv(bbox_path,
                                        delim_whitespace=True,
                                        header=None).astype(int)
        #
        filepath = os.path.join(data_dir, 'CUB_200_2011/images.txt')
        df_filenames = pd.read_csv(filepath, delim_whitespace=True, header=None)
        filenames = df_filenames[1].tolist()
        print('Total filenames: ', len(filenames), filenames[0])
        #
        filename_bbox = {img_file[:-4]: [] for img_file in filenames}
        numImgs = len(filenames)
        for i in range(0, numImgs):
            # bbox = [x-left, y-top, width, height]
            bbox = df_bounding_boxes.iloc[i][1:].tolist()

            key = filenames[i][:-4]
            filename_bbox[key] = bbox
        #
        return filename_bbox

    def load_captions(self, data_dir, filenames):
        all_captions = []
        for i in range(len(filenames)):
            cap_path = '%s/text/%s.txt' % (data_dir, filenames[i])
            with open(cap_path, "r") as f:
                captions = f.read().decode('utf8').split('\n')
                cnt = 0
                for cap in captions:
                    if len(cap) == 0:
                        continue
                    cap = cap.replace("\ufffd\ufffd", " ")
                    # picks out sequences of alphanumeric characters as tokens
                    # and drops everything else
                    tokenizer = RegexpTokenizer(r'\w+')
                    tokens = tokenizer.tokenize(cap.lower())
                    # print('tokens', tokens)
                    if len(tokens) == 0:
                        print('cap', cap)
                        continue

                    tokens_new = []
                    for t in tokens:
                        t = t.encode('ascii', 'ignore').decode('ascii')
                        if len(t) > 0:
                            tokens_new.append(t)
                    all_captions.append(tokens_new)
                    cnt += 1
                    if cnt == self.embeddings_num:
                        break
                if cnt < self.embeddings_num:
                    print('ERROR: the captions for %s less than %d'% (filenames[i], cnt))
        return all_captions

    def build_dictionary(self, train_captions, test_captions):
        word_counts = defaultdict(float)
        captions = train_captions + test_captions
        for sent in captions:
            for word in sent:
                word_counts[word] += 1

        vocab = [w for w in word_counts if word_counts[w] >= 0]

        ixtoword = {}
        ixtoword[0] = '<end>'
        wordtoix = {}
        wordtoix['<end>'] = 0
        ix = 1
        for w in vocab:
            wordtoix[w] = ix
            ixtoword[ix] = w
            ix += 1

        train_captions_new = []
        for t in train_captions:
            rev = []
            for w in t:
                if w in wordtoix:
                    rev.append(wordtoix[w])
            # rev.append(0)  # do not need '<end>' token
            train_captions_new.append(rev)

        test_captions_new = []
        for t in test_captions:
            rev = []
            for w in t:
                if w in wordtoix:
                    rev.append(wordtoix[w])
            # rev.append(0)  # do not need '<end>' token
            test_captions_new.append(rev)

        return [train_captions_new, test_captions_new,
                ixtoword, wordtoix, len(ixtoword)]

    def load_text_data(self, data_dir, split):
        filepath = os.path.join(data_dir, 'captions.pickle')
        train_names = self.load_filenames(data_dir, 'train')
        test_names = self.load_filenames(data_dir, 'test')
        if not os.path.isfile(filepath):
            train_captions = self.load_captions(data_dir, train_names)
            test_captions = self.load_captions(data_dir, test_names)

            train_captions, test_captions, ixtoword, wordtoix, n_words = \
                self.build_dictionary(train_captions, test_captions)
            with open(filepath, 'wb') as f:
                pickle.dump([train_captions, test_captions,
                                ixtoword, wordtoix], f, protocol=2)
                print('Save to: ', filepath)
        else:
            with open(filepath, 'rb') as f:
                x = pickle.load(f)
                train_captions, test_captions = x[0], x[1]
                ixtoword, wordtoix = x[2], x[3]
                del x
                n_words = len(ixtoword)
                print('Load from: ', filepath)
        if split == 'train':
            # a list of list: each list contains
            # the indices of words in a sentence
            captions = train_captions
            filenames = train_names
        else:  # split=='test'
            captions = test_captions
            filenames = test_names
        return filenames, captions, ixtoword, wordtoix, n_words

    def load_class_id(self, data_dir, total_num):
        if os.path.isfile(data_dir + '/class_info.pickle'):
            with open(data_dir + '/class_info.pickle', 'rb') as f:
                class_id = pickle.load(f , encoding = 'latin1')
        else:
            class_id = np.arange(total_num)
        return class_id

    def load_filenames(self, data_dir, split):
        filepath = '%s/%s/filenames.pickle' % (data_dir, split)
        if os.path.isfile(filepath):
            with open(filepath, 'rb') as f:
                filenames = pickle.load(f)
            print('Load filenames from: %s (%d)' % (filepath, len(filenames)))
        else:
            filenames = []
        return filenames

    def get_caption(self, sent_ix):
        # a list of indices for a sentence
        sent_caption = np.asarray(self.captions[sent_ix]).astype('int64')
        if (sent_caption == 0).sum() > 0:
            print('ERROR: do not need END (0) token', sent_caption)
        num_words = len(sent_caption)
        # pad with 0s (i.e., '<end>')
        x = np.zeros((cfg.TEXT.WORDS_NUM, 1), dtype='int64')
        x_len = num_words
        if num_words <= cfg.TEXT.WORDS_NUM:
            x[:num_words, 0] = sent_caption
        else:
            ix = list(np.arange(num_words))  # 1, 2, 3,..., maxNum
            np.random.shuffle(ix)
            ix = ix[:cfg.TEXT.WORDS_NUM]
            ix = np.sort(ix)
            x[:, 0] = sent_caption[ix]
            x_len = cfg.TEXT.WORDS_NUM
        return x, x_len

    def __getitem__(self, index):
        #
        key = self.filenames[index]
        cls_id = self.class_id[index]
        #
        if self.bbox is not None:
            bbox = self.bbox[key]
            data_dir = '%s/CUB_200_2011' % self.data_dir
        else:
            bbox = None
            data_dir = self.data_dir
        #
        img_name = '%s/images/%s.jpg' % (data_dir, key)
        imgs = get_imgs(img_name, self.imsize,
                        bbox, self.transform, normalize=self.norm)
        # random select a sentence
        sent_ix = random.randint(0, self.embeddings_num)
        new_sent_ix = index * self.embeddings_num + sent_ix
        caps, cap_len = self.get_caption(new_sent_ix)

        caps_dec = []
        for word in caps:
            caps_dec.append(self.ixtoword[int(word)])

        return imgs, caps, cap_len, cls_id, key, self.input_ids[index], self.segments_ids[index]


    def __len__(self):
        return len(self.filenames)


class CNN_ENCODER(nn.Module):
    def __init__(self, nef):
        super(CNN_ENCODER, self).__init__()
        if cfg.TRAIN.FLAG:
            self.nef = nef
        else:
            self.nef = 256  # define a uniform ranker
        
        print('CNN_ENCODER')
        model = models.inception_v3()
        print('loaded_inception')
        url = 'https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth'
        model.load_state_dict(model_zoo.load_url(url))
        for param in model.parameters():
            param.requires_grad = False
        print('Load pretrained model from ', url)
        # print(model)

        self.define_module(model)
        self.init_trainable_weights()

    def define_module(self, model):
        self.Conv2d_1a_3x3 = model.Conv2d_1a_3x3
        self.Conv2d_2a_3x3 = model.Conv2d_2a_3x3
        self.Conv2d_2b_3x3 = model.Conv2d_2b_3x3
        self.Conv2d_3b_1x1 = model.Conv2d_3b_1x1
        self.Conv2d_4a_3x3 = model.Conv2d_4a_3x3
        self.Mixed_5b = model.Mixed_5b
        self.Mixed_5c = model.Mixed_5c
        self.Mixed_5d = model.Mixed_5d
        self.Mixed_6a = model.Mixed_6a
        self.Mixed_6b = model.Mixed_6b
        self.Mixed_6c = model.Mixed_6c
        self.Mixed_6d = model.Mixed_6d
        self.Mixed_6e = model.Mixed_6e
        self.Mixed_7a = model.Mixed_7a
        self.Mixed_7b = model.Mixed_7b
        self.Mixed_7c = model.Mixed_7c

        self.emb_features = conv1x1(768, self.nef)
        self.emb_cnn_code = nn.Linear(2048, self.nef)

    def init_trainable_weights(self):
        initrange = 0.1
        self.emb_features.weight.data.uniform_(-initrange, initrange)
        self.emb_cnn_code.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        features = None
        # --> fixed-size input: batch x 3 x 299 x 299
        x = nn.Upsample(size=(299, 299), mode='bilinear')(x)
        # 299 x 299 x 3
        x = self.Conv2d_1a_3x3(x)
        # 149 x 149 x 32
        x = self.Conv2d_2a_3x3(x)
        # 147 x 147 x 32
        x = self.Conv2d_2b_3x3(x)
        # 147 x 147 x 64
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 73 x 73 x 64
        x = self.Conv2d_3b_1x1(x)
        # 73 x 73 x 80
        x = self.Conv2d_4a_3x3(x)
        # 71 x 71 x 192

        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 35 x 35 x 192
        x = self.Mixed_5b(x)
        # 35 x 35 x 256
        x = self.Mixed_5c(x)
        # 35 x 35 x 288
        x = self.Mixed_5d(x)
        # 35 x 35 x 288

        x = self.Mixed_6a(x)
        # 17 x 17 x 768
        x = self.Mixed_6b(x)
        # 17 x 17 x 768
        x = self.Mixed_6c(x)
        # 17 x 17 x 768
        x = self.Mixed_6d(x)
        # 17 x 17 x 768
        x = self.Mixed_6e(x)
        # 17 x 17 x 768

        # image region features
        features = x
        # 17 x 17 x 768

        x = self.Mixed_7a(x)
        # 8 x 8 x 1280
        x = self.Mixed_7b(x)
        # 8 x 8 x 2048
        x = self.Mixed_7c(x)
        # 8 x 8 x 2048
        x = F.avg_pool2d(x, kernel_size=8)
        # 1 x 1 x 2048
        # x = F.dropout(x, training=self.training)
        # 1 x 1 x 2048
        x = x.view(x.size(0), -1)
        # 2048

        # global image features
        cnn_code = self.emb_cnn_code(x)
        # 512
        if features is not None:
            features = self.emb_features(features)
        return features, cnn_code

class BERT_ENCODER(nn.Module):
    def __init__(self):
        super(BERT_ENCODER, self).__init__()

        
        print('Bert_ENCODER')
        
        model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
        for param in model.parameters():
            param.requires_grad = False
        print('Load pretrained model from  BertModel')

        self.define_module(model)
        self.init_trainable_weights()
        self.bert_model = model

    def define_module(self, model):
        self.word_bert_code = nn.Linear(768, 256)
        self.sent_bert_code = nn.Linear(768, 256)

    def init_trainable_weights(self):
        initrange = 0.1
        self.word_bert_code.weight.data.uniform_(-initrange, initrange)
        self.sent_bert_code.weight.data.uniform_(-initrange, initrange)

    def forward(self,  b_input_ids, b_segments_ids):
        
        outputs = self.bert_model(b_input_ids, b_segments_ids)
        hidden_states = outputs[2]

        word_embedding = torch.stack(hidden_states, dim=0)


        word_embedding= self.word_bert_code(word_embedding)


        word_embedding = word_embedding.permute(1,3,0,2)

        word_embedding = word_embedding.sum(dim=2)



        token_vecs = hidden_states[-2]

        # Calculate the average of all 18 token vectors.
        sentence_embedding = torch.mean(token_vecs, dim=1)
        sentence_embedding= self.sent_bert_code(sentence_embedding)


        return  word_embedding, sentence_embedding



def drawCaption(convas, captions, ixtoword, vis_size, off1=2, off2=2):
    
    FONT_MAX = 50

    num = captions.size(0)
    img_txt = Image.fromarray(convas)
    # get a font
    # fnt = None  # ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 50)
    print ("CURRENT WORKING DIRCTORY : " , os.getcwd())
    fnt = ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 50)
    # get a drawing context
    d = ImageDraw.Draw(img_txt)
    sentence_list = []
    for i in range(num):
        cap = captions[i].data.cpu().numpy()
        sentence = []
        for j in range(len(cap)):
            if cap[j] == 0:
                break
            word = ixtoword[cap[j]].encode('ascii', 'ignore').decode('ascii')
            d.text(((j + off1) * (vis_size + off2), i * FONT_MAX), '%d:%s' % (j, word[:6]),
                   font=fnt, fill=(255, 255, 255, 255))
            sentence.append(word)
        sentence_list.append(sentence)
    return img_txt, sentence_list

def cosine_similarity(x1, x2, dim=1, eps=1e-8):
    """Returns cosine similarity between x1 and x2, computed along dim.
    """
    w12 = torch.sum(x1 * x2, dim)
    w1 = torch.norm(x1, 2, dim)
    w2 = torch.norm(x2, 2, dim)
    return (w12 / (w1 * w2).clamp(min=eps)).squeeze()

def sent_loss(cnn_code, rnn_code, labels, class_ids, batch_size, eps=1e-8):
    # ### Mask mis-match samples  ###
    # that come from the same class as the real sample ###
    masks = []
    if class_ids is not None:
        for i in range(batch_size):
            mask = (class_ids == class_ids[i]).astype(np.uint8)
            mask[i] = 0
            masks.append(mask.reshape((1, -1)))
        masks = np.concatenate(masks, 0)
        # masks: batch_size x batch_size
        masks = torch.BoolTensor(masks)
        if cfg.CUDA:
            masks = masks.cuda()

    # --> seq_len x batch_size x nef
    if cnn_code.dim() == 2:
        cnn_code = cnn_code.unsqueeze(0)
        rnn_code = rnn_code.unsqueeze(0)

    # cnn_code_norm / rnn_code_norm: seq_len x batch_size x 1
    cnn_code_norm = torch.norm(cnn_code, 2, dim=2, keepdim=True)
    rnn_code_norm = torch.norm(rnn_code, 2, dim=2, keepdim=True)
    # scores* / norm*: seq_len x batch_size x batch_size
    scores0 = torch.bmm(cnn_code, rnn_code.transpose(1, 2))
    norm0 = torch.bmm(cnn_code_norm, rnn_code_norm.transpose(1, 2))
    scores0 = scores0 / norm0.clamp(min=eps) * cfg.TRAIN.SMOOTH.GAMMA3

    # --> batch_size x batch_size
    scores0 = scores0.squeeze()
    if class_ids is not None:
        scores0.data.masked_fill_(masks, -float('inf'))
    scores1 = scores0.transpose(0, 1)
    if labels is not None:
        loss0 = nn.CrossEntropyLoss()(scores0, labels)
        loss1 = nn.CrossEntropyLoss()(scores1, labels)
    else:
        loss0, loss1 = None, None
    return loss0, loss1


def words_loss(img_features, words_emb, labels, cap_lens, class_ids, batch_size):
    """
        words_emb(query): batch x nef x seq_len
        img_features(context): batch x nef x 17 x 17
    """
    masks = []
    att_maps = []
    similarities = []
    cap_lens = cap_lens.data.tolist()

    for i in range(batch_size):
    
        if class_ids is not None:
            mask = (class_ids == class_ids[i]).astype(np.uint8)
            mask[i] = 0
            masks.append(mask.reshape((1, -1)))
        # Get the i-th text description
        words_num = cap_lens[i]
        # -> 1 x nef x words_num
        word = words_emb[i, :, :words_num].unsqueeze(0).contiguous()
        # -> batch_size x nef x words_num
        word = word.repeat(batch_size, 1, 1)
        # batch x nef x 17*17
        context = img_features
        """
            word(query): batch x nef x words_num
            context: batch x nef x 17 x 17
            weiContext: batch x nef x words_num
            attn: batch x words_num x 17 x 17
        """
        weiContext, attn = func_attention(word, context, cfg.TRAIN.SMOOTH.GAMMA1)
        att_maps.append(attn[i].unsqueeze(0).contiguous())
        # --> batch_size x words_num x nef
        word = word.transpose(1, 2).contiguous()
        weiContext = weiContext.transpose(1, 2).contiguous()
        # --> batch_size*words_num x nef
        word = word.view(batch_size * words_num, -1)
        weiContext = weiContext.view(batch_size * words_num, -1)
        #
        # -->batch_size*words_num
        row_sim = cosine_similarity(word, weiContext)
        # --> batch_size x words_num
        row_sim = row_sim.view(batch_size, words_num)

        # Eq. (10)
        row_sim.mul_(cfg.TRAIN.SMOOTH.GAMMA2).exp_()
        row_sim = row_sim.sum(dim=1, keepdim=True)
        row_sim = torch.log(row_sim)

        # --> 1 x batch_size
        # similarities(i, j): the similarity between the i-th image and the j-th text description
        similarities.append(row_sim)


    # batch_size x batch_size
    similarities = torch.cat(similarities, 1)
    if class_ids is not None:
        masks = np.concatenate(masks, 0)
        # masks: batch_size x batch_size
        masks = torch.BoolTensor(masks)
        if cfg.CUDA:
            masks = masks.cuda()

    similarities = similarities * cfg.TRAIN.SMOOTH.GAMMA3
    if class_ids is not None:
        similarities.data.masked_fill_(masks, -float('inf'))
    similarities1 = similarities.transpose(0, 1)
    if labels is not None:
        loss0 = nn.CrossEntropyLoss()(similarities, labels)
        loss1 = nn.CrossEntropyLoss()(similarities1, labels)
    else:
        loss0, loss1 = None, None
    return loss0, loss1, att_maps

def func_attention(query, context, gamma1):
    """
    query: batch x ndf x queryL
    context: batch x ndf x ih x iw (sourceL=ihxiw)
    mask: batch_size x sourceL
    """
    batch_size, queryL = query.size(0), query.size(2)
    ih, iw = context.size(2), context.size(3)
    sourceL = ih * iw

    # --> batch x sourceL x ndf
    context = context.view(batch_size, -1, sourceL)
    contextT = torch.transpose(context, 1, 2).contiguous()

    # Get attention
    # (batch x sourceL x ndf)(batch x ndf x queryL)
    # -->batch x sourceL x queryL
    attn = torch.bmm(contextT, query) # Eq. (7) in AttnGAN paper
    # --> batch*sourceL x queryL
    attn = attn.view(batch_size*sourceL, queryL)
    attn = nn.Softmax()(attn)  # Eq. (8)

    # --> batch x sourceL x queryL
    attn = attn.view(batch_size, sourceL, queryL)
    # --> batch*queryL x sourceL
    attn = torch.transpose(attn, 1, 2).contiguous()
    attn = attn.view(batch_size*queryL, sourceL)
    #  Eq. (9)
    attn = attn * gamma1
    attn = nn.Softmax()(attn)
    attn = attn.view(batch_size, queryL, sourceL)
    # --> batch x sourceL x queryL
    attnT = torch.transpose(attn, 1, 2).contiguous()

    # (batch x ndf x sourceL)(batch x sourceL x queryL)
    # --> batch x ndf x queryL
    weightedContext = torch.bmm(context, attnT)

    return weightedContext, attn.view(batch_size, -1, ih, iw)


def train(dataloader, cnn_model, bert_encoder, batch_size, labels, optimizer, epoch, ixtoword, image_dir):
    train_function_start_time = time.time()
    cnn_model.train() #Sets the module in training mode.
    #rnn_model.train() #Sets the module in training mode.
    bert_encoder.train()
    
    s_total_loss0 = 0
    s_total_loss1 = 0
    w_total_loss0 = 0
    w_total_loss1 = 0
    
    # print("keyword |||||||||||||||||||||||||||||||")
    # print("len(dataloader) : " , len(dataloader) )
    # print(" count = " ,  (epoch + 1) * len(dataloader)  )
    # print("keyword |||||||||||||||||||||||||||||||")
    count = (epoch + 1) * len(dataloader)
    start_time = time.time()


    for step, data in enumerate(dataloader, 0):
    
    
        # Loading the first batch (number of batches/steps in an epoch is 183)
        #rnn_model.zero_grad()
        cnn_model.zero_grad()

        imgs, captions, cap_lens, class_ids, keys, b_input_ids, b_segments_ids = prepare_data(data)


        # words_features: batch_size x 256 x 17 x 17 ==> # image region features
        # sent_code: batch_size x 256                ==> # global image features
        words_features, sent_code = cnn_model(imgs[-1])
        # --> batch_size x nef x 17*17
        nef, att_sze = words_features.size(1), words_features.size(2)# 256, 17(16th of the whole image)
        # words_features = words_features.view(batch_size, nef, -1)

        #hidden = rnn_model.init_hidden(batch_size) # A tuple of 2 zero tensor of torch.Size([2, 48, 128])
        # words_emb: batch_size x nef x seq_len
        # sent_emb: batch_size x nef
        #words_emb, sent_emb = rnn_model(captions, cap_lens, hidden)

        #-------------------------------------------------------------------------------
                #---------------------------------------------------------
                        #-----------------------------------
        words_emb, sent_emb =bert_encoder(b_input_ids, b_segments_ids)
                        #-----------------------------------
                #---------------------------------------------------------
        #-------------------------------------------------------------------------------

        w_loss0, w_loss1, attn_maps = words_loss(words_features, words_emb, labels, cap_lens, class_ids, batch_size)
        w_total_loss0 += w_loss0.data
        w_total_loss1 += w_loss1.data
        loss = w_loss0 + w_loss1

        s_loss0, s_loss1 = sent_loss(sent_code, sent_emb, labels, class_ids, batch_size)
        loss += s_loss0 + s_loss1
        s_total_loss0 += s_loss0.data
        s_total_loss1 += s_loss1.data
        #
        loss.backward()
        #
        # `clip_grad_norm` helps prevent
        # the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm(bert_encoder.parameters(), cfg.TRAIN.RNN_GRAD_CLIP)
        optimizer.step()

        if step % UPDATE_INTERVAL == 0:
            count = epoch * len(dataloader) + step

            # print ("====================================================")
            # print ("s_total_loss0 : " , s_total_loss0)
            # print ("s_total_loss0.item() : " , s_total_loss0.item())
            # print ("UPDATE_INTERVAL : " , UPDATE_INTERVAL)
            print ("s_total_loss0.item()/UPDATE_INTERVAL : " , s_total_loss0.item()/UPDATE_INTERVAL)
            print ("s_total_loss1.item()/UPDATE_INTERVAL : " , s_total_loss1.item()/UPDATE_INTERVAL)
            print ("w_total_loss0.item()/UPDATE_INTERVAL : " , w_total_loss0.item()/UPDATE_INTERVAL)
            print ("w_total_loss1.item()/UPDATE_INTERVAL : " , w_total_loss1.item()/UPDATE_INTERVAL)
            # print ("s_total_loss0/UPDATE_INTERVAL : " , s_total_loss0/UPDATE_INTERVAL)
            # print ("=====================================================")
            s_cur_loss0 = s_total_loss0.item() / UPDATE_INTERVAL
            s_cur_loss1 = s_total_loss1.item() / UPDATE_INTERVAL

            w_cur_loss0 = w_total_loss0.item() / UPDATE_INTERVAL
            w_cur_loss1 = w_total_loss1.item() / UPDATE_INTERVAL

            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | '
                    's_loss {:5.2f} {:5.2f} | '
                    'w_loss {:5.2f} {:5.2f}'
                    .format(epoch, step, len(dataloader),
                          elapsed * 1000. / UPDATE_INTERVAL,
                            s_cur_loss0, s_cur_loss1,
                            w_cur_loss0, w_cur_loss1))
            s_total_loss0 = 0
            s_total_loss1 = 0
            w_total_loss0 = 0
            w_total_loss1 = 0
            start_time = time.time()
            # attention Maps
            #Save image only every 8 epochs && Save it to The Drive
            if (epoch % 8 == 0):
                print("bulding images")
                img_set, _ = build_super_images(imgs[-1].cpu(), captions, ixtoword, attn_maps, att_sze)
                if img_set is not None:
                    im = Image.fromarray(img_set)
                    fullpath = '%s/attention_maps%d.png' % (image_dir, step)
                    im.save(fullpath)
                    mydriveimg = '/content/drive/My Drive/BertCUBImage'
                    drivepath = '%s/attention_maps%d.png' % (mydriveimg, epoch)
                    im.save(drivepath)
    print("keyTime |||||||||||||||||||||||||||||||")
    print("train_function_time : " , time.time() - train_function_start_time)
    print("KeyTime |||||||||||||||||||||||||||||||")
    return count


def evaluate(dataloader, cnn_model, bert_encoder, batch_size):
    cnn_model.eval()
    bert_encoder.eval()
    s_total_loss = 0
    w_total_loss = 0
    for step, data in enumerate(dataloader, 0):
        real_imgs, captions, cap_lens, class_ids, keys, b_input_ids, b_segments_ids = prepare_data(data)

        words_features, sent_code = cnn_model(real_imgs[-1])
        # nef = words_features.size(1)
        # words_features = words_features.view(batch_size, nef, -1)
        words_emb, sent_emb =bert_encoder(b_input_ids, b_segments_ids)



        w_loss0, w_loss1, attn = words_loss(words_features, words_emb, labels, cap_lens, class_ids, batch_size)
        w_total_loss += (w_loss0 + w_loss1).data

        s_loss0, s_loss1 = sent_loss(sent_code, sent_emb, labels, class_ids, batch_size)
        s_total_loss += (s_loss0 + s_loss1).data

        if step == 50:
            break

    s_cur_loss = s_total_loss.item() / step
    w_cur_loss = w_total_loss.item() / step

    return s_cur_loss, w_cur_loss


def build_models():
    # build model ############################################################
    #text_encoder = RNN_ENCODER(dataset.n_words, nhidden=cfg.TEXT.EMBEDDING_DIM)
    '''
    RNN_ENCODER(
    (encoder): Embedding(5450, 300)
    (drop): Dropout(p=0.5, inplace=False)
    (rnn): LSTM(300, 128, batch_first=True, dropout=0.5, bidirectional=True))
    '''
    print('build_models')
    image_encoder = CNN_ENCODER(cfg.TEXT.EMBEDDING_DIM)
    bert_encoder = BERT_ENCODER()

    labels = Variable(torch.LongTensor(range(batch_size)))
    '''
    A tensor of [0,1,2,3,...,47]
    '''
    start_epoch = 0
    if cfg.TRAIN.NET_E != '':
        state_dict = torch.load(cfg.TRAIN.NET_E)
        bert_encoder.load_state_dict(state_dict)
        print('Load ', cfg.TRAIN.NET_E)
        #
        name = cfg.TRAIN.NET_E.replace('text_encoder', 'image_encoder')
        state_dict = torch.load(name)
        image_encoder.load_state_dict(state_dict)
        print('Load ', name)

        istart = cfg.TRAIN.NET_E.rfind('_') + 8
        iend = cfg.TRAIN.NET_E.rfind('.')
        start_epoch = cfg.TRAIN.NET_E[istart:iend]
        start_epoch = int(start_epoch) + 1
        print('start_epoch', start_epoch)
    if cfg.CUDA:
        #text_encoder = text_encoder.cuda()
        image_encoder = image_encoder.cuda()
        labels = labels.cuda()
        bert_encoder = bert_encoder.cuda()

    return bert_encoder, image_encoder, labels, start_epoch


__name__ = "__main__"
if __name__ == "__main__":
    print('Using config:')
    pprint.pprint(cfg)

    UPDATE_INTERVAL = 200

    ##########################################################################
    now = datetime.datetime.now(dateutil.tz.tzlocal())
    timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
    output_dir = '../output/%s_%s_%s' % (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

    model_dir = os.path.join(output_dir, 'Model')
    image_dir = os.path.join(output_dir, 'Image')
    mkdir_p(model_dir)
    mkdir_p(image_dir)

    torch.cuda.set_device(cfg.GPU_ID)
    cudnn.benchmark = True

    # Get data loader ##################################################
    imsize = 299
    batch_size = 48

    image_transform = transforms.Compose([transforms.Scale(355), transforms.RandomCrop(imsize), transforms.RandomHorizontalFlip()])
    
    dataset = TextDataset(cfg.DATA_DIR, 'train', base_size=cfg.TREE.BASE_SIZE, transform=image_transform, input_ids=input_ids, segments_ids=segments_ids, sentences=sentences )
    print(dataset.n_words, dataset.embeddings_num)
    assert dataset

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=int(cfg.WORKERS))
    #using prepare data functiont this dataloader yieldes:
    #imgs: a list of 1 tensor of size torch.Size([48, 3, 299, 299])
    #captons: a  tensor of size torch.Size([48, 18]), shorter filled with end words converted by word to index
    #cap_lens: a  tensor of size torch.Size([48]) , acual caps lens order from big to small (max is 18)
    #class_ids: a 48 ints list in range 0-200 of the classes
    #keys: a 48 string list  of the classes classes nammes crosspondening to the class_ids
    

    # # validation data #
    dataset_val = TextDataset(cfg.DATA_DIR, 'test', base_size=cfg.TREE.BASE_SIZE,transform=image_transform, input_ids=input_ids, segments_ids=segments_ids, sentences=sentences)
    print(dataset_val.n_words, dataset_val.embeddings_num)
    assert dataset_val
    dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, drop_last=True,shuffle=True, num_workers=int(cfg.WORKERS))

    # Train ##############################################################
    bert_encoder, image_encoder, labels, start_epoch = build_models()
    para = []
    
    for v in bert_encoder.parameters(): # 4 parameters
        if v.requires_grad:
            para.append(v)
    
    for v in image_encoder.parameters(): # 3 parameters
        if v.requires_grad:
            para.append(v)

    print ('requires_grad =' , len(para))
    # optimizer = optim.Adam(para, lr=cfg.TRAIN.ENCODER_LR, betas=(0.5, 0.999))
    # At any point you can hit Ctrl + C to break out of training early.

    try:
        lr = cfg.TRAIN.ENCODER_LR #0.002
        print("keyword |||||||||||||||||||||||||||||||")
        print("Start_epoch : " , start_epoch)
        print("cfg.TRAIN.MAX_EPOCH : " , cfg.TRAIN.MAX_EPOCH )
        print("keyword |||||||||||||||||||||||||||||||")


        for epoch in range(start_epoch, cfg.TRAIN.MAX_EPOCH):
            
            one_epoch_start_time = time.time()
            optimizer = optim.Adam(para, lr=lr, betas=(0.5, 0.999))
            epoch_start_time = time.time()
            count = train(dataloader, image_encoder, bert_encoder, batch_size, labels, optimizer, epoch, dataset.ixtoword, image_dir)
            print('-' * 89)
            if len(dataloader_val) > 0:
                s_loss, w_loss = evaluate(dataloader_val, image_encoder, bert_encoder, batch_size)
                print('| end epoch {:3d} | valid loss ''{:5.2f} {:5.2f} | lr {:.5f}|'.format(epoch, s_loss, w_loss, lr))
            print('-' * 89)
            if lr > 0.0002 : #cfg.TRAIN.ENCODER_LR/10.:
                lr *= 0.98

            print("keyTime |||||||||||||||||||||||||||||||")
            print("one_epoch_time : " , time.time() - one_epoch_start_time)
            print("KeyTime |||||||||||||||||||||||||||||||")

            if (epoch % 8 == 0 or epoch == cfg.TRAIN.MAX_EPOCH or epoch == cfg.TRAIN.MAX_EPOCH-1 ):
                mydrivemodel = '/content/drive/My Drive/BertCUBModel'
                torch.save(image_encoder.state_dict(), '%s/image_encoder%d.pth' % (model_dir, epoch))
                torch.save(image_encoder.state_dict(), '%s/image_encoder%d.pth' % (mydrivemodel, epoch))
                torch.save(bert_encoder.state_dict(), '%s/text_encoder%d.pth' % (model_dir, epoch))
                torch.save(bert_encoder.state_dict(), '%s/text_encoder%d.pth' % (mydrivemodel, epoch))
                print('Save G/Ds models.')


    except KeyboardInterrupt:
        print('-' * 89)
        print('Exiting from training early')



Using config:
{'B_VALIDATION': False,
 'CONFIG_NAME': 'DAMSM',
 'CUDA': True,
 'DATASET_NAME': 'birds',
 'DATA_DIR': '../data/birds',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'GF_DIM': 128,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': 0,
 'TEXT': {'CAPTIONS_PER_IMAGE': 10, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 18},
 'TRAIN': {'BATCH_SIZE': 48,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 600,
           'NET_E': '',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 1.0},
           'SNAPSHOT_INTERVAL': 50},
 'TREE': {'BASE_SIZE': 299, 'BRANCH_NUM': 1},
 'WORKERS': 1}
self.imsize [299]
Total filenames:  11788 001.Bla

/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:211: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


Load filenames from: ../data/birds/train/filenames.pickle (8855)
Load filenames from: ../data/birds/test/filenames.pickle (2933)
Load from:  ../data/birds/captions.pickle
5450 10
self.imsize [299]
Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: ../data/birds/train/filenames.pickle (8855)
Load filenames from: ../data/birds/test/filenames.pickle (2933)
Load from:  ../data/birds/captions.pickle
5450 10
requires_grad = 7
keyword |||||||||||||||||||||||||||||||
Start_epoch :  0
cfg.TRAIN.MAX_EPOCH :  600
keyword |||||||||||||||||||||||||||||||


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:978: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:987: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1062: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


s_total_loss0.item()/UPDATE_INTERVAL :  0.0194236421585083
s_total_loss1.item()/UPDATE_INTERVAL :  0.019812434911727905
w_total_loss0.item()/UPDATE_INTERVAL :  0.023000130653381346
w_total_loss1.item()/UPDATE_INTERVAL :  0.019997397661209105
| epoch   0 |     0/  184 batches | ms/batch  5.13 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  216.07643914222717
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  322.2639937400818
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   0 | valid loss  7.86  7.87 | lr 0.00200|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  351.15519285202026
KeyTime |||||||||||||||||||||||||||||||
S

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:288: RuntimeWarning: invalid value encountered in true_divide


keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  214.91586756706238
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  316.3087911605835
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   8 | valid loss  7.86  7.86 | lr 0.00170|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  342.2672095298767
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019312115907669066
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931214451789856
w_total_loss0.item()/UPDATE_INTERVAL :  0.019318814277648925
w_total_loss1.item()/UPDATE_INTERVAL :  0.019317636489868163
| epoch   9 |     0/  184 batches | ms/batch  5.31 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_fu

In [29]:
Using config:
{'B_VALIDATION': False,
 'CONFIG_NAME': 'DAMSM',
 'CUDA': True,
 'DATASET_NAME': 'birds',
 'DATA_DIR': '../data/birds',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'GF_DIM': 128,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': 0,
 'TEXT': {'CAPTIONS_PER_IMAGE': 10, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 18},
 'TRAIN': {'BATCH_SIZE': 48,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 600,
           'NET_E': '',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 1.0},
           'SNAPSHOT_INTERVAL': 50},
 'TREE': {'BASE_SIZE': 299, 'BRANCH_NUM': 1},
 'WORKERS': 1}
self.imsize [299]
Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:211: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
Load filenames from: ../data/birds/train/filenames.pickle (8855)
Load filenames from: ../data/birds/test/filenames.pickle (2933)
Load from:  ../data/birds/captions.pickle
5450 10
self.imsize [299]
Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: ../data/birds/train/filenames.pickle (8855)
Load filenames from: ../data/birds/test/filenames.pickle (2933)
Load from:  ../data/birds/captions.pickle
5450 10
requires_grad = 7
keyword |||||||||||||||||||||||||||||||
Start_epoch :  0
cfg.TRAIN.MAX_EPOCH :  600
keyword |||||||||||||||||||||||||||||||
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:978: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:987: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1062: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
s_total_loss0.item()/UPDATE_INTERVAL :  0.0194236421585083
s_total_loss1.item()/UPDATE_INTERVAL :  0.019812434911727905
w_total_loss0.item()/UPDATE_INTERVAL :  0.023000130653381346
w_total_loss1.item()/UPDATE_INTERVAL :  0.019997397661209105
| epoch   0 |     0/  184 batches | ms/batch  5.13 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  216.07643914222717
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  322.2639937400818
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   0 | valid loss  7.86  7.87 | lr 0.00200|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  351.15519285202026
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019337291717529296
s_total_loss1.item()/UPDATE_INTERVAL :  0.019335851669311524
w_total_loss0.item()/UPDATE_INTERVAL :  0.019371705055236818
w_total_loss1.item()/UPDATE_INTERVAL :  0.01936876654624939
| epoch   1 |     0/  184 batches | ms/batch  5.37 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.88432002067566
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   1 | valid loss  7.86  7.86 | lr 0.00196|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.84982919692993
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019327635765075683
s_total_loss1.item()/UPDATE_INTERVAL :  0.019327906370162965
w_total_loss0.item()/UPDATE_INTERVAL :  0.019284191131591796
w_total_loss1.item()/UPDATE_INTERVAL :  0.019280529022216795
| epoch   2 |     0/  184 batches | ms/batch  4.58 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.30532217025757
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   2 | valid loss  7.86  7.86 | lr 0.00192|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  124.34788966178894
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019340577125549315
s_total_loss1.item()/UPDATE_INTERVAL :  0.019341064691543577
w_total_loss0.item()/UPDATE_INTERVAL :  0.019303005933761597
w_total_loss1.item()/UPDATE_INTERVAL :  0.019302825927734375
| epoch   3 |     0/  184 batches | ms/batch  4.52 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.77145624160767
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   3 | valid loss  7.86  7.86 | lr 0.00188|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.50538754463196
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019343637228012085
s_total_loss1.item()/UPDATE_INTERVAL :  0.01934399127960205
w_total_loss0.item()/UPDATE_INTERVAL :  0.019304075241088868
w_total_loss1.item()/UPDATE_INTERVAL :  0.019304269552230836
| epoch   4 |     0/  184 batches | ms/batch  4.61 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.33532524108887
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   4 | valid loss  7.86  7.86 | lr 0.00184|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  124.26443433761597
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01926831841468811
s_total_loss1.item()/UPDATE_INTERVAL :  0.019268797636032106
w_total_loss0.item()/UPDATE_INTERVAL :  0.019260042905807497
w_total_loss1.item()/UPDATE_INTERVAL :  0.019259427785873414
| epoch   5 |     0/  184 batches | ms/batch  4.64 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.06309390068054
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   5 | valid loss  7.86  7.86 | lr 0.00181|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.61632895469666
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931622505187988
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931655764579773
w_total_loss0.item()/UPDATE_INTERVAL :  0.019339613914489746
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933781981468201
| epoch   6 |     0/  184 batches | ms/batch  4.58 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.66865277290344
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   6 | valid loss  7.86  7.86 | lr 0.00177|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.18379592895508
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019291883707046507
s_total_loss1.item()/UPDATE_INTERVAL :  0.019292136430740358
w_total_loss0.item()/UPDATE_INTERVAL :  0.019308449029922486
w_total_loss1.item()/UPDATE_INTERVAL :  0.019307832717895507
| epoch   7 |     0/  184 batches | ms/batch  4.63 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.73477077484131
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   7 | valid loss  7.86  7.86 | lr 0.00174|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.54203367233276
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019348506927490235
s_total_loss1.item()/UPDATE_INTERVAL :  0.019348464012145995
w_total_loss0.item()/UPDATE_INTERVAL :  0.019364967346191406
w_total_loss1.item()/UPDATE_INTERVAL :  0.01936126708984375
| epoch   8 |     0/  184 batches | ms/batch  4.55 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:288: RuntimeWarning: invalid value encountered in true_divide
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  214.91586756706238
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  316.3087911605835
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   8 | valid loss  7.86  7.86 | lr 0.00170|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  342.2672095298767
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019312115907669066
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931214451789856
w_total_loss0.item()/UPDATE_INTERVAL :  0.019318814277648925
w_total_loss1.item()/UPDATE_INTERVAL :  0.019317636489868163
| epoch   9 |     0/  184 batches | ms/batch  5.31 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  99.6510283946991
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch   9 | valid loss  7.86  7.86 | lr 0.00167|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.11780500411987
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019371219873428346
s_total_loss1.item()/UPDATE_INTERVAL :  0.019371277093887328
w_total_loss0.item()/UPDATE_INTERVAL :  0.019350744485855102
w_total_loss1.item()/UPDATE_INTERVAL :  0.019350506067276
| epoch  10 |     0/  184 batches | ms/batch  4.79 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.73433804512024
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  10 | valid loss  7.85  7.85 | lr 0.00163|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.89917707443237
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019358978271484376
s_total_loss1.item()/UPDATE_INTERVAL :  0.019359060525894166
w_total_loss0.item()/UPDATE_INTERVAL :  0.01938998818397522
w_total_loss1.item()/UPDATE_INTERVAL :  0.0193893563747406
| epoch  11 |     0/  184 batches | ms/batch  4.87 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.94185590744019
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  11 | valid loss  7.86  7.86 | lr 0.00160|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  131.60406351089478
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933971881866455
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933996319770813
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932379722595215
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323204755783082
| epoch  12 |     0/  184 batches | ms/batch  4.94 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  104.4150595664978
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  12 | valid loss  7.86  7.86 | lr 0.00157|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  131.61839199066162
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019334572553634643
s_total_loss1.item()/UPDATE_INTERVAL :  0.019334577322006226
w_total_loss0.item()/UPDATE_INTERVAL :  0.019303492307662963
w_total_loss1.item()/UPDATE_INTERVAL :  0.01930339217185974
| epoch  13 |     0/  184 batches | ms/batch  4.96 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.01550960540771
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  13 | valid loss  7.86  7.85 | lr 0.00154|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  128.57114219665527
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019315894842147827
s_total_loss1.item()/UPDATE_INTERVAL :  0.019315942525863647
w_total_loss0.item()/UPDATE_INTERVAL :  0.019333189725875853
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933309555053711
| epoch  14 |     0/  184 batches | ms/batch  4.84 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.00803518295288
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  14 | valid loss  7.86  7.86 | lr 0.00151|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.17169690132141
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019293256998062135
s_total_loss1.item()/UPDATE_INTERVAL :  0.019293235540390016
w_total_loss0.item()/UPDATE_INTERVAL :  0.019329482316970827
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329322576522825
| epoch  15 |     0/  184 batches | ms/batch  4.87 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  99.33703112602234
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  15 | valid loss  7.86  7.86 | lr 0.00148|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  125.24967575073242
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019329928159713745
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932991862297058
w_total_loss0.item()/UPDATE_INTERVAL :  0.019342690706253052
w_total_loss1.item()/UPDATE_INTERVAL :  0.019342440366744994
| epoch  16 |     0/  184 batches | ms/batch  4.74 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  216.7243902683258
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  318.16154623031616
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  16 | valid loss  7.86  7.86 | lr 0.00145|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  343.9111976623535
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019361478090286256
s_total_loss1.item()/UPDATE_INTERVAL :  0.019361554384231566
w_total_loss0.item()/UPDATE_INTERVAL :  0.019347820281982422
w_total_loss1.item()/UPDATE_INTERVAL :  0.019347724914550782
| epoch  17 |     0/  184 batches | ms/batch  5.26 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.09342527389526
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  17 | valid loss  7.86  7.85 | lr 0.00142|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.83542132377625
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193511164188385
s_total_loss1.item()/UPDATE_INTERVAL :  0.019351080656051636
w_total_loss0.item()/UPDATE_INTERVAL :  0.019347209930419922
w_total_loss1.item()/UPDATE_INTERVAL :  0.019346905946731566
| epoch  18 |     0/  184 batches | ms/batch  4.70 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.24635553359985
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  18 | valid loss  7.86  7.86 | lr 0.00139|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.89225769042969
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932884693145752
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932881474494934
w_total_loss0.item()/UPDATE_INTERVAL :  0.019322251081466676
w_total_loss1.item()/UPDATE_INTERVAL :  0.019322353601455688
| epoch  19 |     0/  184 batches | ms/batch  4.64 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.09172987937927
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  19 | valid loss  7.86  7.86 | lr 0.00136|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.7549934387207
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019317082166671752
s_total_loss1.item()/UPDATE_INTERVAL :  0.019317116737365723
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933700084686279
w_total_loss1.item()/UPDATE_INTERVAL :  0.019336758852005004
| epoch  20 |     0/  184 batches | ms/batch  4.66 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.18931722640991
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  20 | valid loss  7.86  7.86 | lr 0.00134|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.68663382530212
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019297715425491333
s_total_loss1.item()/UPDATE_INTERVAL :  0.019297704696655274
w_total_loss0.item()/UPDATE_INTERVAL :  0.019308418035507202
w_total_loss1.item()/UPDATE_INTERVAL :  0.01930782675743103
| epoch  21 |     0/  184 batches | ms/batch  4.71 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.16626024246216
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  21 | valid loss  7.86  7.86 | lr 0.00131|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.82688522338867
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931129813194275
s_total_loss1.item()/UPDATE_INTERVAL :  0.019311304092407226
w_total_loss0.item()/UPDATE_INTERVAL :  0.01935497760772705
w_total_loss1.item()/UPDATE_INTERVAL :  0.01935479521751404
| epoch  22 |     0/  184 batches | ms/batch  4.65 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.95625638961792
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  22 | valid loss  7.86  7.86 | lr 0.00128|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.81795859336853
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01937169671058655
s_total_loss1.item()/UPDATE_INTERVAL :  0.01937167525291443
w_total_loss0.item()/UPDATE_INTERVAL :  0.019337854385375976
w_total_loss1.item()/UPDATE_INTERVAL :  0.019337464570999146
| epoch  23 |     0/  184 batches | ms/batch  4.74 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.35873746871948
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  23 | valid loss  7.86  7.86 | lr 0.00126|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  124.05032992362976
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930232882499695
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930224061012268
w_total_loss0.item()/UPDATE_INTERVAL :  0.019296979904174803
w_total_loss1.item()/UPDATE_INTERVAL :  0.01929726004600525
| epoch  24 |     0/  184 batches | ms/batch  4.67 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  214.280748128891
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  315.1752986907959
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  24 | valid loss  7.86  7.86 | lr 0.00123|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  341.10942125320435
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931104302406311
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931121826171875
w_total_loss0.item()/UPDATE_INTERVAL :  0.019321856498718263
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321774244308473
| epoch  25 |     0/  184 batches | ms/batch  5.34 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.30909371376038
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  25 | valid loss  7.86  7.86 | lr 0.00121|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.9569206237793
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019297356605529784
s_total_loss1.item()/UPDATE_INTERVAL :  0.019297326803207396
w_total_loss0.item()/UPDATE_INTERVAL :  0.019293107986450196
w_total_loss1.item()/UPDATE_INTERVAL :  0.01929293990135193
| epoch  26 |     0/  184 batches | ms/batch  4.67 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.35682606697083
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  26 | valid loss  7.86  7.86 | lr 0.00118|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.28349471092224
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019325613975524902
s_total_loss1.item()/UPDATE_INTERVAL :  0.019325693845748903
w_total_loss0.item()/UPDATE_INTERVAL :  0.019285759925842284
w_total_loss1.item()/UPDATE_INTERVAL :  0.01928518891334534
| epoch  27 |     0/  184 batches | ms/batch  4.78 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.36207818984985
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  27 | valid loss  7.86  7.86 | lr 0.00116|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  124.07548713684082
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01929419994354248
s_total_loss1.item()/UPDATE_INTERVAL :  0.01929415702819824
w_total_loss0.item()/UPDATE_INTERVAL :  0.019320292472839354
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319262504577637
| epoch  28 |     0/  184 batches | ms/batch  4.64 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.55140256881714
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  28 | valid loss  7.86  7.86 | lr 0.00114|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.3636724948883
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932076096534729
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320894479751587
w_total_loss0.item()/UPDATE_INTERVAL :  0.019351099729537965
w_total_loss1.item()/UPDATE_INTERVAL :  0.019350546598434448
| epoch  29 |     0/  184 batches | ms/batch  4.76 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.6625235080719
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  29 | valid loss  7.86  7.86 | lr 0.00111|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  125.01987624168396
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930716633796692
s_total_loss1.item()/UPDATE_INTERVAL :  0.019307169914245605
w_total_loss0.item()/UPDATE_INTERVAL :  0.019330564737319946
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329854249954224
| epoch  30 |     0/  184 batches | ms/batch  4.80 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  99.9193172454834
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  30 | valid loss  7.86  7.86 | lr 0.00109|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.29921102523804
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019313498735427856
s_total_loss1.item()/UPDATE_INTERVAL :  0.019313615560531617
w_total_loss0.item()/UPDATE_INTERVAL :  0.019312733411788942
w_total_loss1.item()/UPDATE_INTERVAL :  0.019312127828598022
| epoch  31 |     0/  184 batches | ms/batch  4.67 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.61511707305908
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  31 | valid loss  7.86  7.86 | lr 0.00107|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  127.1897087097168
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932364821434021
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932368755340576
w_total_loss0.item()/UPDATE_INTERVAL :  0.019298597574234008
w_total_loss1.item()/UPDATE_INTERVAL :  0.01929817199707031
| epoch  32 |     0/  184 batches | ms/batch  4.71 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  218.21341967582703
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  321.99297046661377
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  32 | valid loss  7.86  7.86 | lr 0.00105|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  348.78967905044556
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019305723905563354
s_total_loss1.item()/UPDATE_INTERVAL :  0.019305700063705446
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933146595954895
w_total_loss1.item()/UPDATE_INTERVAL :  0.019331413507461547
| epoch  33 |     0/  184 batches | ms/batch  5.54 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.36748576164246
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  33 | valid loss  7.86  7.86 | lr 0.00103|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  127.83661603927612
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019326448440551758
s_total_loss1.item()/UPDATE_INTERVAL :  0.019326475858688356
w_total_loss0.item()/UPDATE_INTERVAL :  0.019314799308776855
w_total_loss1.item()/UPDATE_INTERVAL :  0.019314885139465332
| epoch  34 |     0/  184 batches | ms/batch  4.83 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.35951232910156
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  34 | valid loss  7.86  7.86 | lr 0.00101|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  128.4821789264679
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019329205751419068
s_total_loss1.item()/UPDATE_INTERVAL :  0.019329134225845337
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931187629699707
w_total_loss1.item()/UPDATE_INTERVAL :  0.019311394691467285
| epoch  35 |     0/  184 batches | ms/batch  4.94 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.50115823745728
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  35 | valid loss  7.86  7.86 | lr 0.00099|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  128.05466294288635
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193049955368042
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930493950843811
w_total_loss0.item()/UPDATE_INTERVAL :  0.019332457780838013
w_total_loss1.item()/UPDATE_INTERVAL :  0.019332195520401
| epoch  36 |     0/  184 batches | ms/batch  4.83 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.393146276474
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  36 | valid loss  7.86  7.86 | lr 0.00097|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  127.25274467468262
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019351406097412108
s_total_loss1.item()/UPDATE_INTERVAL :  0.019351426362991333
w_total_loss0.item()/UPDATE_INTERVAL :  0.019332033395767213
w_total_loss1.item()/UPDATE_INTERVAL :  0.019331878423690795
| epoch  37 |     0/  184 batches | ms/batch  4.96 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.26317501068115
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  37 | valid loss  7.86  7.86 | lr 0.00095|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  127.87749671936035
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019336161613464357
s_total_loss1.item()/UPDATE_INTERVAL :  0.019336090087890626
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931355834007263
w_total_loss1.item()/UPDATE_INTERVAL :  0.019312766790390016
| epoch  38 |     0/  184 batches | ms/batch  5.04 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.67045879364014
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  38 | valid loss  7.86  7.86 | lr 0.00093|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.98074746131897
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931540608406067
s_total_loss1.item()/UPDATE_INTERVAL :  0.019315437078475953
w_total_loss0.item()/UPDATE_INTERVAL :  0.019314533472061156
w_total_loss1.item()/UPDATE_INTERVAL :  0.019314340353012084
| epoch  39 |     0/  184 batches | ms/batch  4.91 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.04520440101624
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  39 | valid loss  7.86  7.86 | lr 0.00091|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.7213785648346
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019345542192459108
s_total_loss1.item()/UPDATE_INTERVAL :  0.019345550537109374
w_total_loss0.item()/UPDATE_INTERVAL :  0.019360953569412233
w_total_loss1.item()/UPDATE_INTERVAL :  0.019359850883483888
| epoch  40 |     0/  184 batches | ms/batch  4.95 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  216.91792845726013
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  321.41070008277893
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  40 | valid loss  7.86  7.86 | lr 0.00089|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  348.3623869419098
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019337759017944336
s_total_loss1.item()/UPDATE_INTERVAL :  0.019337724447250366
w_total_loss0.item()/UPDATE_INTERVAL :  0.019319101572036743
w_total_loss1.item()/UPDATE_INTERVAL :  0.019318705797195433
| epoch  41 |     0/  184 batches | ms/batch  5.81 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.8307855129242
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  41 | valid loss  7.86  7.86 | lr 0.00087|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.0397732257843
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019330681562423707
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933070182800293
w_total_loss0.item()/UPDATE_INTERVAL :  0.019312061071395874
w_total_loss1.item()/UPDATE_INTERVAL :  0.019311861991882326
| epoch  42 |     0/  184 batches | ms/batch  5.17 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.89825487136841
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  42 | valid loss  7.86  7.86 | lr 0.00086|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.19805002212524
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933456540107727
s_total_loss1.item()/UPDATE_INTERVAL :  0.019334577322006226
w_total_loss0.item()/UPDATE_INTERVAL :  0.019334983825683594
w_total_loss1.item()/UPDATE_INTERVAL :  0.019334717988967895
| epoch  43 |     0/  184 batches | ms/batch  4.96 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  104.62281250953674
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  43 | valid loss  7.86  7.86 | lr 0.00084|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  132.17799592018127
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019280033111572267
s_total_loss1.item()/UPDATE_INTERVAL :  0.019279913902282717
w_total_loss0.item()/UPDATE_INTERVAL :  0.019301023483276367
w_total_loss1.item()/UPDATE_INTERVAL :  0.019299768209457398
| epoch  44 |     0/  184 batches | ms/batch  5.19 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  104.08975338935852
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  44 | valid loss  7.86  7.86 | lr 0.00082|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  131.76607871055603
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019289836883544922
s_total_loss1.item()/UPDATE_INTERVAL :  0.019289716482162475
w_total_loss0.item()/UPDATE_INTERVAL :  0.019287198781967163
w_total_loss1.item()/UPDATE_INTERVAL :  0.019287176132202148
| epoch  45 |     0/  184 batches | ms/batch  5.09 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  106.2483880519867
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  45 | valid loss  7.86  7.86 | lr 0.00081|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  134.12465643882751
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932236075401306
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932233214378357
w_total_loss0.item()/UPDATE_INTERVAL :  0.019334224462509157
w_total_loss1.item()/UPDATE_INTERVAL :  0.019334311485290526
| epoch  46 |     0/  184 batches | ms/batch  5.11 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  105.88632106781006
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  46 | valid loss  7.86  7.86 | lr 0.00079|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  133.877210855484
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019382486343383788
s_total_loss1.item()/UPDATE_INTERVAL :  0.01938252329826355
w_total_loss0.item()/UPDATE_INTERVAL :  0.0193559467792511
w_total_loss1.item()/UPDATE_INTERVAL :  0.019355539083480835
| epoch  47 |     0/  184 batches | ms/batch  5.18 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  106.94004106521606
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  47 | valid loss  7.86  7.86 | lr 0.00077|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  135.08276963233948
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193213951587677
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932135462760925
w_total_loss0.item()/UPDATE_INTERVAL :  0.019324451684951782
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323835372924803
| epoch  48 |     0/  184 batches | ms/batch  5.15 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  236.75548267364502
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  343.6794855594635
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  48 | valid loss  7.86  7.86 | lr 0.00076|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  370.9863476753235
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193278968334198
s_total_loss1.item()/UPDATE_INTERVAL :  0.019327934980392456
w_total_loss0.item()/UPDATE_INTERVAL :  0.019354829788208006
w_total_loss1.item()/UPDATE_INTERVAL :  0.01935369610786438
| epoch  49 |     0/  184 batches | ms/batch  5.65 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  104.04374837875366
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  49 | valid loss  7.86  7.86 | lr 0.00074|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  131.29191088676453
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019341444969177245
s_total_loss1.item()/UPDATE_INTERVAL :  0.019341402053833008
w_total_loss0.item()/UPDATE_INTERVAL :  0.019330077171325684
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329758882522582
| epoch  50 |     0/  184 batches | ms/batch  4.99 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.11793279647827
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  50 | valid loss  7.86  7.86 | lr 0.00073|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.10514569282532
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019311795234680174
s_total_loss1.item()/UPDATE_INTERVAL :  0.019311684370040893
w_total_loss0.item()/UPDATE_INTERVAL :  0.019305047988891603
w_total_loss1.item()/UPDATE_INTERVAL :  0.019304631948471068
| epoch  51 |     0/  184 batches | ms/batch  5.04 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.74152302742004
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  51 | valid loss  7.86  7.86 | lr 0.00071|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.82313704490662
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019324727058410644
s_total_loss1.item()/UPDATE_INTERVAL :  0.019324721097946168
w_total_loss0.item()/UPDATE_INTERVAL :  0.019307492971420287
w_total_loss1.item()/UPDATE_INTERVAL :  0.01930771827697754
| epoch  52 |     0/  184 batches | ms/batch  5.06 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.3908064365387
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  52 | valid loss  7.86  7.86 | lr 0.00070|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.5216932296753
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019330724477767944
s_total_loss1.item()/UPDATE_INTERVAL :  0.019330743551254272
w_total_loss0.item()/UPDATE_INTERVAL :  0.019349565505981447
w_total_loss1.item()/UPDATE_INTERVAL :  0.01934937834739685
| epoch  53 |     0/  184 batches | ms/batch  5.10 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.60112285614014
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  53 | valid loss  7.86  7.86 | lr 0.00069|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.69141960144043
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933303236961365
s_total_loss1.item()/UPDATE_INTERVAL :  0.019333167076110838
w_total_loss0.item()/UPDATE_INTERVAL :  0.019348481893539427
w_total_loss1.item()/UPDATE_INTERVAL :  0.019348307847976684
| epoch  54 |     0/  184 batches | ms/batch  5.02 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.10991048812866
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  54 | valid loss  7.86  7.86 | lr 0.00067|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.3489203453064
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019322227239608764
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932221531867981
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328418970108032
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328130483627318
| epoch  55 |     0/  184 batches | ms/batch  5.12 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.59119176864624
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  55 | valid loss  7.86  7.86 | lr 0.00066|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.80328798294067
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01929092764854431
s_total_loss1.item()/UPDATE_INTERVAL :  0.019290844202041624
w_total_loss0.item()/UPDATE_INTERVAL :  0.01927723526954651
w_total_loss1.item()/UPDATE_INTERVAL :  0.01927655339241028
| epoch  56 |     0/  184 batches | ms/batch  5.02 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  231.49682521820068
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  336.5785779953003
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  56 | valid loss  7.86  7.86 | lr 0.00065|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  363.4036693572998
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193289840221405
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328938722610475
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932769179344177
w_total_loss1.item()/UPDATE_INTERVAL :  0.019327555894851686
| epoch  57 |     0/  184 batches | ms/batch  5.62 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.54804801940918
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  57 | valid loss  7.86  7.86 | lr 0.00063|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.18042254447937
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019337323904037477
s_total_loss1.item()/UPDATE_INTERVAL :  0.019337453842163087
w_total_loss0.item()/UPDATE_INTERVAL :  0.019334872961044313
w_total_loss1.item()/UPDATE_INTERVAL :  0.019334346055984497
| epoch  58 |     0/  184 batches | ms/batch  4.94 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.11924338340759
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  58 | valid loss  7.86  7.86 | lr 0.00062|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  127.60801959037781
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019324212074279784
s_total_loss1.item()/UPDATE_INTERVAL :  0.019324140548706056
w_total_loss0.item()/UPDATE_INTERVAL :  0.019330039024353027
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329843521118165
| epoch  59 |     0/  184 batches | ms/batch  4.89 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.57419228553772
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  59 | valid loss  7.86  7.86 | lr 0.00061|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  128.59081482887268
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320521354675293
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320549964904784
w_total_loss0.item()/UPDATE_INTERVAL :  0.019325281381607055
w_total_loss1.item()/UPDATE_INTERVAL :  0.019325567483901976
| epoch  60 |     0/  184 batches | ms/batch  5.02 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.19024538993835
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  60 | valid loss  7.86  7.86 | lr 0.00060|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.8021445274353
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019312649965286255
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931262493133545
w_total_loss0.item()/UPDATE_INTERVAL :  0.019299687147140504
w_total_loss1.item()/UPDATE_INTERVAL :  0.01929859519004822
| epoch  61 |     0/  184 batches | ms/batch  5.27 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  107.92595148086548
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  61 | valid loss  7.86  7.86 | lr 0.00058|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  136.15043783187866
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320887327194215
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932090163230896
w_total_loss0.item()/UPDATE_INTERVAL :  0.019310942888259887
w_total_loss1.item()/UPDATE_INTERVAL :  0.019310550689697267
| epoch  62 |     0/  184 batches | ms/batch  5.01 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  104.22797393798828
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  62 | valid loss  7.86  7.86 | lr 0.00057|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  131.5835108757019
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019299840927124022
s_total_loss1.item()/UPDATE_INTERVAL :  0.019299869537353517
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932433485984802
w_total_loss1.item()/UPDATE_INTERVAL :  0.019324089288711547
| epoch  63 |     0/  184 batches | ms/batch  4.87 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.7205114364624
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  63 | valid loss  7.86  7.86 | lr 0.00056|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.48852825164795
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019323272705078123
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932340979576111
w_total_loss0.item()/UPDATE_INTERVAL :  0.019312511682510376
w_total_loss1.item()/UPDATE_INTERVAL :  0.019312024116516113
| epoch  64 |     0/  184 batches | ms/batch  5.05 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  227.38657665252686
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  332.5835540294647
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  64 | valid loss  7.86  7.86 | lr 0.00055|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  359.6747930049896
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019321876764297485
s_total_loss1.item()/UPDATE_INTERVAL :  0.019321876764297485
w_total_loss0.item()/UPDATE_INTERVAL :  0.019325594902038574
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931710362434387
| epoch  65 |     0/  184 batches | ms/batch  5.78 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.55163097381592
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  65 | valid loss  7.86  7.86 | lr 0.00054|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.70842671394348
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01934354305267334
s_total_loss1.item()/UPDATE_INTERVAL :  0.01934353232383728
w_total_loss0.item()/UPDATE_INTERVAL :  0.019343370199203493
w_total_loss1.item()/UPDATE_INTERVAL :  0.01934289813041687
| epoch  66 |     0/  184 batches | ms/batch  5.03 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.31991267204285
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  66 | valid loss  7.86  7.86 | lr 0.00053|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.36492347717285
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019319299459457397
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931920051574707
w_total_loss0.item()/UPDATE_INTERVAL :  0.019331707954406738
w_total_loss1.item()/UPDATE_INTERVAL :  0.019331221580505372
| epoch  67 |     0/  184 batches | ms/batch  4.98 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.5754451751709
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  67 | valid loss  7.86  7.86 | lr 0.00052|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.72750186920166
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933184027671814
s_total_loss1.item()/UPDATE_INTERVAL :  0.019331773519515993
w_total_loss0.item()/UPDATE_INTERVAL :  0.019323463439941405
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932329773902893
| epoch  68 |     0/  184 batches | ms/batch  4.97 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.40400552749634
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  68 | valid loss  7.86  7.86 | lr 0.00051|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.441734790802
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019313746690750123
s_total_loss1.item()/UPDATE_INTERVAL :  0.019313751459121703
w_total_loss0.item()/UPDATE_INTERVAL :  0.019316807985305787
w_total_loss1.item()/UPDATE_INTERVAL :  0.019316177368164062
| epoch  69 |     0/  184 batches | ms/batch  4.89 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.54749274253845
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  69 | valid loss  7.86  7.86 | lr 0.00050|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.6106698513031
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019321775436401366
s_total_loss1.item()/UPDATE_INTERVAL :  0.019321789741516115
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930214524269104
w_total_loss1.item()/UPDATE_INTERVAL :  0.019301477670669556
| epoch  70 |     0/  184 batches | ms/batch  5.00 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.80286574363708
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  70 | valid loss  7.86  7.86 | lr 0.00049|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  131.04862213134766
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931801199913025
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931803107261658
w_total_loss0.item()/UPDATE_INTERVAL :  0.019314297437667848
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931413769721985
| epoch  71 |     0/  184 batches | ms/batch  4.97 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  104.13133883476257
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  71 | valid loss  7.86  7.86 | lr 0.00048|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  131.28369092941284
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019308300018310548
s_total_loss1.item()/UPDATE_INTERVAL :  0.019308334589004515
w_total_loss0.item()/UPDATE_INTERVAL :  0.019312812089920042
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931234836578369
| epoch  72 |     0/  184 batches | ms/batch  4.92 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  230.7511863708496
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  336.24910402297974
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  72 | valid loss  7.86  7.86 | lr 0.00047|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  363.30855798721313
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019307688474655152
s_total_loss1.item()/UPDATE_INTERVAL :  0.019307669401168823
w_total_loss0.item()/UPDATE_INTERVAL :  0.019325876235961915
w_total_loss1.item()/UPDATE_INTERVAL :  0.019325150251388548
| epoch  73 |     0/  184 batches | ms/batch  5.84 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.01504778862
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  73 | valid loss  7.86  7.86 | lr 0.00046|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.00850868225098
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019308539628982543
s_total_loss1.item()/UPDATE_INTERVAL :  0.019308427572250365
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931015372276306
w_total_loss1.item()/UPDATE_INTERVAL :  0.019309862852096557
| epoch  74 |     0/  184 batches | ms/batch  4.98 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.14033770561218
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  74 | valid loss  7.86  7.86 | lr 0.00045|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  128.96975326538086
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019305481910705566
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930550217628479
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328618049621583
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328206777572632
| epoch  75 |     0/  184 batches | ms/batch  4.87 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.29049706459045
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  75 | valid loss  7.86  7.86 | lr 0.00044|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.3938705921173
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193444287776947
s_total_loss1.item()/UPDATE_INTERVAL :  0.019344396591186523
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934632658958435
w_total_loss1.item()/UPDATE_INTERVAL :  0.019346104860305788
| epoch  76 |     0/  184 batches | ms/batch  5.03 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.70517897605896
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  76 | valid loss  7.86  7.86 | lr 0.00043|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.6390619277954
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019349044561386107
s_total_loss1.item()/UPDATE_INTERVAL :  0.019349018335342406
w_total_loss0.item()/UPDATE_INTERVAL :  0.019347327947616576
w_total_loss1.item()/UPDATE_INTERVAL :  0.019346814155578613
| epoch  77 |     0/  184 batches | ms/batch  4.89 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.01082229614258
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  77 | valid loss  7.86  7.85 | lr 0.00042|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.979731798172
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019332064390182493
s_total_loss1.item()/UPDATE_INTERVAL :  0.019332088232040405
w_total_loss0.item()/UPDATE_INTERVAL :  0.019327011108398438
w_total_loss1.item()/UPDATE_INTERVAL :  0.019326635599136353
| epoch  78 |     0/  184 batches | ms/batch  4.91 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.28992986679077
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  78 | valid loss  7.86  7.86 | lr 0.00041|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.97622537612915
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019341014623641968
s_total_loss1.item()/UPDATE_INTERVAL :  0.01934094786643982
w_total_loss0.item()/UPDATE_INTERVAL :  0.019353212118148805
w_total_loss1.item()/UPDATE_INTERVAL :  0.01935301423072815
| epoch  79 |     0/  184 batches | ms/batch  4.93 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.93363833427429
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  79 | valid loss  7.86  7.86 | lr 0.00041|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  131.29498767852783
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01927780270576477
s_total_loss1.item()/UPDATE_INTERVAL :  0.01927779793739319
w_total_loss0.item()/UPDATE_INTERVAL :  0.019279179573059083
w_total_loss1.item()/UPDATE_INTERVAL :  0.01927880883216858
| epoch  80 |     0/  184 batches | ms/batch  4.95 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  224.9109001159668
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  327.4383590221405
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  80 | valid loss  7.86  7.86 | lr 0.00040|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  354.03486490249634
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019306960105895995
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193069326877594
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930332660675049
w_total_loss1.item()/UPDATE_INTERVAL :  0.019302810430526732
| epoch  81 |     0/  184 batches | ms/batch  5.56 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.64856839179993
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  81 | valid loss  7.86  7.86 | lr 0.00039|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.82213592529297
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019326545000076294
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932658314704895
w_total_loss0.item()/UPDATE_INTERVAL :  0.019311890602111817
w_total_loss1.item()/UPDATE_INTERVAL :  0.019311624765396117
| epoch  82 |     0/  184 batches | ms/batch  4.88 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  99.82880663871765
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  82 | valid loss  7.86  7.86 | lr 0.00038|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.05620217323303
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193110191822052
s_total_loss1.item()/UPDATE_INTERVAL :  0.019311046600341795
w_total_loss0.item()/UPDATE_INTERVAL :  0.019316061735153198
w_total_loss1.item()/UPDATE_INTERVAL :  0.019315758943557738
| epoch  83 |     0/  184 batches | ms/batch  4.74 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  99.15674161911011
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  83 | valid loss  7.86  7.86 | lr 0.00037|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  125.75158333778381
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019310103654861452
s_total_loss1.item()/UPDATE_INTERVAL :  0.019310131072998046
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931984066963196
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319467544555664
| epoch  84 |     0/  184 batches | ms/batch  4.82 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.4675190448761
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  84 | valid loss  7.86  7.86 | lr 0.00037|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.74331974983215
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019311370849609374
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931138038635254
w_total_loss0.item()/UPDATE_INTERVAL :  0.019319733381271364
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931920051574707
| epoch  85 |     0/  184 batches | ms/batch  4.89 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.7518458366394
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  85 | valid loss  7.86  7.86 | lr 0.00036|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  127.12245488166809
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019361332654953003
s_total_loss1.item()/UPDATE_INTERVAL :  0.019361315965652465
w_total_loss0.item()/UPDATE_INTERVAL :  0.019343719482421876
w_total_loss1.item()/UPDATE_INTERVAL :  0.019343433380126954
| epoch  86 |     0/  184 batches | ms/batch  4.75 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.17433738708496
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  86 | valid loss  7.86  7.86 | lr 0.00035|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.47674822807312
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019334816932678224
s_total_loss1.item()/UPDATE_INTERVAL :  0.019334802627563475
w_total_loss0.item()/UPDATE_INTERVAL :  0.019347013235092164
w_total_loss1.item()/UPDATE_INTERVAL :  0.019346574544906615
| epoch  87 |     0/  184 batches | ms/batch  4.91 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.15701293945312
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  87 | valid loss  7.85  7.85 | lr 0.00034|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.69148659706116
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019323701858520507
s_total_loss1.item()/UPDATE_INTERVAL :  0.019323749542236326
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932268023490906
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932252287864685
| epoch  88 |     0/  184 batches | ms/batch  4.94 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  224.0284161567688
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  328.1854202747345
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  88 | valid loss  7.86  7.86 | lr 0.00034|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  354.98670864105225
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019311228990554808
s_total_loss1.item()/UPDATE_INTERVAL :  0.019311223030090332
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328346252441408
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932814598083496
| epoch  89 |     0/  184 batches | ms/batch  5.58 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.147705078125
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  89 | valid loss  7.86  7.86 | lr 0.00033|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  128.9968466758728
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930479407310486
s_total_loss1.item()/UPDATE_INTERVAL :  0.019304791688919066
w_total_loss0.item()/UPDATE_INTERVAL :  0.019311579465866088
w_total_loss1.item()/UPDATE_INTERVAL :  0.019311432838439942
| epoch  90 |     0/  184 batches | ms/batch  4.90 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.41316723823547
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  90 | valid loss  7.86  7.86 | lr 0.00032|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  128.14429235458374
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019324837923049926
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932484984397888
w_total_loss0.item()/UPDATE_INTERVAL :  0.019329047203063963
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328240156173706
| epoch  91 |     0/  184 batches | ms/batch  4.83 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.0581283569336
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  91 | valid loss  7.85  7.85 | lr 0.00032|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  127.886971950531
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019309345483779907
s_total_loss1.item()/UPDATE_INTERVAL :  0.019309425354003908
w_total_loss0.item()/UPDATE_INTERVAL :  0.019300462007522584
w_total_loss1.item()/UPDATE_INTERVAL :  0.019300156831741334
| epoch  92 |     0/  184 batches | ms/batch  4.88 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.04044961929321
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  92 | valid loss  7.86  7.86 | lr 0.00031|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  127.72946572303772
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019328216314315794
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932819724082947
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933631896972656
w_total_loss1.item()/UPDATE_INTERVAL :  0.019336031675338747
| epoch  93 |     0/  184 batches | ms/batch  5.00 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.29595613479614
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  93 | valid loss  7.86  7.86 | lr 0.00031|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  127.96769571304321
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019312490224838257
s_total_loss1.item()/UPDATE_INTERVAL :  0.019312548637390136
w_total_loss0.item()/UPDATE_INTERVAL :  0.019310528039932252
w_total_loss1.item()/UPDATE_INTERVAL :  0.019309755563735962
| epoch  94 |     0/  184 batches | ms/batch  4.97 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.39470529556274
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  94 | valid loss  7.86  7.86 | lr 0.00030|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  128.12292337417603
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01928419589996338
s_total_loss1.item()/UPDATE_INTERVAL :  0.01928421139717102
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930853009223938
w_total_loss1.item()/UPDATE_INTERVAL :  0.019308229684829713
| epoch  95 |     0/  184 batches | ms/batch  4.92 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.44919991493225
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  95 | valid loss  7.86  7.86 | lr 0.00029|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.2719132900238
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.0193340003490448
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933406949043274
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933503270149231
w_total_loss1.item()/UPDATE_INTERVAL :  0.019334970712661742
| epoch  96 |     0/  184 batches | ms/batch  4.95 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  227.44567966461182
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  332.4896514415741
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  96 | valid loss  7.86  7.86 | lr 0.00029|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  359.4129283428192
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933151125907898
s_total_loss1.item()/UPDATE_INTERVAL :  0.019331493377685548
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931281924247742
w_total_loss1.item()/UPDATE_INTERVAL :  0.019312723875045776
| epoch  97 |     0/  184 batches | ms/batch  5.68 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.96822428703308
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  97 | valid loss  7.86  7.86 | lr 0.00028|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  128.63808965682983
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019335943460464477
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933595061302185
w_total_loss0.item()/UPDATE_INTERVAL :  0.019339603185653687
w_total_loss1.item()/UPDATE_INTERVAL :  0.019339284896850585
| epoch  98 |     0/  184 batches | ms/batch  4.96 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.10246896743774
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  98 | valid loss  7.86  7.85 | lr 0.00028|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  131.64282369613647
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933947205543518
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933946132659912
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933605670928955
w_total_loss1.item()/UPDATE_INTERVAL :  0.019335594177246094
| epoch  99 |     0/  184 batches | ms/batch  5.25 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  104.44430422782898
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch  99 | valid loss  7.86  7.86 | lr 0.00027|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  132.34212732315063
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931949257850647
s_total_loss1.item()/UPDATE_INTERVAL :  0.019319421052932738
w_total_loss0.item()/UPDATE_INTERVAL :  0.019295076131820677
w_total_loss1.item()/UPDATE_INTERVAL :  0.019294754266738892
| epoch 100 |     0/  184 batches | ms/batch  4.83 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  104.18543672561646
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 100 | valid loss  7.86  7.86 | lr 0.00027|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  131.3818097114563
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932461142539978
s_total_loss1.item()/UPDATE_INTERVAL :  0.019324636459350585
w_total_loss0.item()/UPDATE_INTERVAL :  0.01934110164642334
w_total_loss1.item()/UPDATE_INTERVAL :  0.019341014623641968
| epoch 101 |     0/  184 batches | ms/batch  5.00 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.82299065589905
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 101 | valid loss  7.85  7.85 | lr 0.00026|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.95490336418152
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930622577667236
s_total_loss1.item()/UPDATE_INTERVAL :  0.019306248426437377
w_total_loss0.item()/UPDATE_INTERVAL :  0.019309955835342407
w_total_loss1.item()/UPDATE_INTERVAL :  0.019309765100479125
| epoch 102 |     0/  184 batches | ms/batch  5.05 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  103.33289885520935
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 102 | valid loss  7.86  7.86 | lr 0.00025|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  130.4823100566864
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933611273765564
s_total_loss1.item()/UPDATE_INTERVAL :  0.019336127042770386
w_total_loss0.item()/UPDATE_INTERVAL :  0.019333282709121703
w_total_loss1.item()/UPDATE_INTERVAL :  0.019333033561706542
| epoch 103 |     0/  184 batches | ms/batch  5.07 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.67555665969849
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 103 | valid loss  7.86  7.86 | lr 0.00025|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.429669380188
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019342550039291383
s_total_loss1.item()/UPDATE_INTERVAL :  0.019342541694641113
w_total_loss0.item()/UPDATE_INTERVAL :  0.019333573579788207
w_total_loss1.item()/UPDATE_INTERVAL :  0.019333410263061523
| epoch 104 |     0/  184 batches | ms/batch  4.92 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  224.48705625534058
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  328.8051509857178
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 104 | valid loss  7.86  7.86 | lr 0.00024|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  355.7792556285858
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932873249053955
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328724145889282
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932977318763733
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329061508178712
| epoch 105 |     0/  184 batches | ms/batch  5.56 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  102.63926434516907
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 105 | valid loss  7.86  7.86 | lr 0.00024|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.4452259540558
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933815598487854
s_total_loss1.item()/UPDATE_INTERVAL :  0.019338165521621705
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932698369026184
w_total_loss1.item()/UPDATE_INTERVAL :  0.0193264639377594
| epoch 106 |     0/  184 batches | ms/batch  4.94 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.22801280021667
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 106 | valid loss  7.86  7.86 | lr 0.00023|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.68494939804077
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019330973625183104
s_total_loss1.item()/UPDATE_INTERVAL :  0.019330930709838868
w_total_loss0.item()/UPDATE_INTERVAL :  0.019337421655654906
w_total_loss1.item()/UPDATE_INTERVAL :  0.019336700439453125
| epoch 107 |     0/  184 batches | ms/batch  4.79 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  100.34360551834106
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 107 | valid loss  7.86  7.86 | lr 0.00023|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.65800356864929
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933185935020447
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933184504508972
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932778239250183
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932664752006531
| epoch 108 |     0/  184 batches | ms/batch  4.74 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  99.98073053359985
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 108 | valid loss  7.86  7.86 | lr 0.00023|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  126.11475872993469
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019317702054977418
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931766390800476
w_total_loss0.item()/UPDATE_INTERVAL :  0.019322576522827147
w_total_loss1.item()/UPDATE_INTERVAL :  0.019322031736373903
| epoch 109 |     0/  184 batches | ms/batch  4.72 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  99.27503108978271
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 109 | valid loss  7.86  7.86 | lr 0.00022|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  125.44038224220276
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019327048063278198
s_total_loss1.item()/UPDATE_INTERVAL :  0.019327079057693482
w_total_loss0.item()/UPDATE_INTERVAL :  0.019342331886291503
w_total_loss1.item()/UPDATE_INTERVAL :  0.019341996908187865
| epoch 110 |     0/  184 batches | ms/batch  4.90 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.87796521186829
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 110 | valid loss  7.86  7.86 | lr 0.00022|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  125.12661910057068
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019329910278320313
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932987689971924
w_total_loss0.item()/UPDATE_INTERVAL :  0.019321528673172
w_total_loss1.item()/UPDATE_INTERVAL :  0.019321112632751464
| epoch 111 |     0/  184 batches | ms/batch  4.78 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  101.77620840072632
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 111 | valid loss  7.86  7.86 | lr 0.00021|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  129.09556031227112
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01935125231742859
s_total_loss1.item()/UPDATE_INTERVAL :  0.01935123562812805
w_total_loss0.item()/UPDATE_INTERVAL :  0.019328765869140625
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328361749649046
| epoch 112 |     0/  184 batches | ms/batch  4.96 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  221.9969449043274
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  323.6003484725952
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 112 | valid loss  7.86  7.86 | lr 0.00021|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  349.62456130981445
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019315391778945923
s_total_loss1.item()/UPDATE_INTERVAL :  0.019315428733825683
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931271195411682
w_total_loss1.item()/UPDATE_INTERVAL :  0.019312127828598022
| epoch 113 |     0/  184 batches | ms/batch  5.71 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  99.71046376228333
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 113 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  125.7139663696289
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019332698583602904
s_total_loss1.item()/UPDATE_INTERVAL :  0.019332680702209473
w_total_loss0.item()/UPDATE_INTERVAL :  0.019330285787582398
w_total_loss1.item()/UPDATE_INTERVAL :  0.019329466819763184
| epoch 114 |     0/  184 batches | ms/batch  4.69 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.36122274398804
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 114 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  124.25367426872253
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019317893981933593
s_total_loss1.item()/UPDATE_INTERVAL :  0.019317930936813353
w_total_loss0.item()/UPDATE_INTERVAL :  0.0193165385723114
w_total_loss1.item()/UPDATE_INTERVAL :  0.019316120147705076
| epoch 115 |     0/  184 batches | ms/batch  4.80 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.17445468902588
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 115 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  124.04331254959106
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931458830833435
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193146550655365
w_total_loss0.item()/UPDATE_INTERVAL :  0.019317482709884645
w_total_loss1.item()/UPDATE_INTERVAL :  0.019317337274551392
| epoch 116 |     0/  184 batches | ms/batch  4.92 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  98.1266348361969
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 116 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.89443778991699
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01930393695831299
s_total_loss1.item()/UPDATE_INTERVAL :  0.019303892850875855
w_total_loss0.item()/UPDATE_INTERVAL :  0.01931474208831787
w_total_loss1.item()/UPDATE_INTERVAL :  0.019314414262771605
| epoch 117 |     0/  184 batches | ms/batch  4.82 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.73175239562988
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 117 | valid loss  7.86  7.85 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.60949850082397
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019331378936767576
s_total_loss1.item()/UPDATE_INTERVAL :  0.019331411123275757
w_total_loss0.item()/UPDATE_INTERVAL :  0.019348406791687013
w_total_loss1.item()/UPDATE_INTERVAL :  0.019347801208496093
| epoch 118 |     0/  184 batches | ms/batch  4.63 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.93422174453735
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 118 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.61012101173401
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019327648878097535
s_total_loss1.item()/UPDATE_INTERVAL :  0.0193275785446167
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932819724082947
w_total_loss1.item()/UPDATE_INTERVAL :  0.019327692985534668
| epoch 119 |     0/  184 batches | ms/batch  4.72 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.83646488189697
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 119 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.54484629631042
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933450222015381
s_total_loss1.item()/UPDATE_INTERVAL :  0.019334468841552734
w_total_loss0.item()/UPDATE_INTERVAL :  0.019319413900375365
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319282770156862
| epoch 120 |     0/  184 batches | ms/batch  4.76 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  215.5310001373291
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  315.79226994514465
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 120 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  341.5713264942169
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019313721656799315
s_total_loss1.item()/UPDATE_INTERVAL :  0.019313703775405883
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930505394935608
w_total_loss1.item()/UPDATE_INTERVAL :  0.0193048894405365
| epoch 121 |     0/  184 batches | ms/batch  5.41 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.9155216217041
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 121 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.56322193145752
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019300185441970825
s_total_loss1.item()/UPDATE_INTERVAL :  0.019300132989883423
w_total_loss0.item()/UPDATE_INTERVAL :  0.01928592324256897
w_total_loss1.item()/UPDATE_INTERVAL :  0.019285916090011596
| epoch 122 |     0/  184 batches | ms/batch  4.89 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.10618305206299
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 122 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.79444551467896
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019326934814453123
s_total_loss1.item()/UPDATE_INTERVAL :  0.019326976537704467
w_total_loss0.item()/UPDATE_INTERVAL :  0.01930403709411621
w_total_loss1.item()/UPDATE_INTERVAL :  0.019303574562072753
| epoch 123 |     0/  184 batches | ms/batch  4.73 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.32376194000244
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 123 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.00350213050842
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932835578918457
s_total_loss1.item()/UPDATE_INTERVAL :  0.019328376054763795
w_total_loss0.item()/UPDATE_INTERVAL :  0.0193098509311676
w_total_loss1.item()/UPDATE_INTERVAL :  0.0193094539642334
| epoch 124 |     0/  184 batches | ms/batch  4.62 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.32798266410828
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 124 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.88319540023804
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019323829412460327
s_total_loss1.item()/UPDATE_INTERVAL :  0.019323803186416626
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932887554168701
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328765869140625
| epoch 125 |     0/  184 batches | ms/batch  4.64 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.09142756462097
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 125 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.65162920951843
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320569038391112
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320573806762695
w_total_loss0.item()/UPDATE_INTERVAL :  0.019333853721618652
w_total_loss1.item()/UPDATE_INTERVAL :  0.01933182716369629
| epoch 126 |     0/  184 batches | ms/batch  4.77 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.19191455841064
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 126 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.76757478713989
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931768298149109
s_total_loss1.item()/UPDATE_INTERVAL :  0.01931766629219055
w_total_loss0.item()/UPDATE_INTERVAL :  0.019332109689712523
w_total_loss1.item()/UPDATE_INTERVAL :  0.019331773519515993
| epoch 127 |     0/  184 batches | ms/batch  4.63 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.19590282440186
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 127 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.75714945793152
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019321675300598144
s_total_loss1.item()/UPDATE_INTERVAL :  0.019321727752685546
w_total_loss0.item()/UPDATE_INTERVAL :  0.019300999641418456
w_total_loss1.item()/UPDATE_INTERVAL :  0.019300861358642576
| epoch 128 |     0/  184 batches | ms/batch  4.68 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  215.35766339302063
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  314.7950496673584
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 128 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  340.5110192298889
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.01933444380760193
s_total_loss1.item()/UPDATE_INTERVAL :  0.01933441638946533
w_total_loss0.item()/UPDATE_INTERVAL :  0.019322911500930785
w_total_loss1.item()/UPDATE_INTERVAL :  0.019322308301925658
| epoch 129 |     0/  184 batches | ms/batch  5.46 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.42383885383606
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 129 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.99108505249023
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019339374303817748
s_total_loss1.item()/UPDATE_INTERVAL :  0.019339370727539062
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932712197303772
w_total_loss1.item()/UPDATE_INTERVAL :  0.019326714277267457
| epoch 130 |     0/  184 batches | ms/batch  4.68 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.38559627532959
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 130 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.97559571266174
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019285508394241334
s_total_loss1.item()/UPDATE_INTERVAL :  0.019285560846328736
w_total_loss0.item()/UPDATE_INTERVAL :  0.019296188354492188
w_total_loss1.item()/UPDATE_INTERVAL :  0.019295879602432252
| epoch 131 |     0/  184 batches | ms/batch  4.70 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.32028722763062
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 131 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.09740662574768
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019325037002563477
s_total_loss1.item()/UPDATE_INTERVAL :  0.019325047731399536
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932857871055603
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932822823524475
| epoch 132 |     0/  184 batches | ms/batch  4.74 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.55124998092651
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 132 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.07505631446838
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01932942509651184
s_total_loss1.item()/UPDATE_INTERVAL :  0.019329375028610228
w_total_loss0.item()/UPDATE_INTERVAL :  0.01933255195617676
w_total_loss1.item()/UPDATE_INTERVAL :  0.019332383871078492
| epoch 133 |     0/  184 batches | ms/batch  4.70 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.32659912109375
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 133 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.00325107574463
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019309722185134888
s_total_loss1.item()/UPDATE_INTERVAL :  0.019309625625610352
w_total_loss0.item()/UPDATE_INTERVAL :  0.019324041604995727
w_total_loss1.item()/UPDATE_INTERVAL :  0.019323660135269163
| epoch 134 |     0/  184 batches | ms/batch  4.71 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.5239326953888
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 134 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.11484384536743
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019326428174972533
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932642102241516
w_total_loss0.item()/UPDATE_INTERVAL :  0.019339362382888792
w_total_loss1.item()/UPDATE_INTERVAL :  0.019339011907577516
| epoch 135 |     0/  184 batches | ms/batch  4.72 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.56098294258118
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 135 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.12460732460022
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019320141077041626
s_total_loss1.item()/UPDATE_INTERVAL :  0.019320119619369507
w_total_loss0.item()/UPDATE_INTERVAL :  0.019322925806045534
w_total_loss1.item()/UPDATE_INTERVAL :  0.019322677850723266
| epoch 136 |     0/  184 batches | ms/batch  4.84 | s_loss  0.02  0.02 | w_loss  0.02  0.02
bulding images
CURRENT WORKING DIRCTORY :  /content/AttnGAN/code
keyTime |||||||||||||||||||||||||||||||
build_super_images_time :  215.4054594039917
KeyTime |||||||||||||||||||||||||||||||
keyTime |||||||||||||||||||||||||||||||
train_function_time :  315.1673033237457
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 136 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  340.9038016796112
KeyTime |||||||||||||||||||||||||||||||
Save G/Ds models.
s_total_loss0.item()/UPDATE_INTERVAL :  0.019323874711990357
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932388424873352
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932029366493225
w_total_loss1.item()/UPDATE_INTERVAL :  0.019319874048233033
| epoch 137 |     0/  184 batches | ms/batch  5.37 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.65082812309265
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 137 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.40971875190735
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01934099316596985
s_total_loss1.item()/UPDATE_INTERVAL :  0.019341005086898802
w_total_loss0.item()/UPDATE_INTERVAL :  0.019346128702163696
w_total_loss1.item()/UPDATE_INTERVAL :  0.019345651865005493
| epoch 138 |     0/  184 batches | ms/batch  4.82 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.20678544044495
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 138 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.73614835739136
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019347654581069948
s_total_loss1.item()/UPDATE_INTERVAL :  0.01934766173362732
w_total_loss0.item()/UPDATE_INTERVAL :  0.01932931900024414
w_total_loss1.item()/UPDATE_INTERVAL :  0.019328761100769042
| epoch 139 |     0/  184 batches | ms/batch  4.83 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.14013409614563
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 139 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  122.8936219215393
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019300512075424194
s_total_loss1.item()/UPDATE_INTERVAL :  0.01930045962333679
w_total_loss0.item()/UPDATE_INTERVAL :  0.019299367666244505
w_total_loss1.item()/UPDATE_INTERVAL :  0.019299193620681762
| epoch 140 |     0/  184 batches | ms/batch  4.81 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.79055905342102
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 140 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.34701681137085
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.01931821823120117
s_total_loss1.item()/UPDATE_INTERVAL :  0.019318188428878783
w_total_loss0.item()/UPDATE_INTERVAL :  0.019320001602172853
w_total_loss1.item()/UPDATE_INTERVAL :  0.01931939721107483
| epoch 141 |     0/  184 batches | ms/batch  4.68 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.46791505813599
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 141 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.20658230781555
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019321125745773316
s_total_loss1.item()/UPDATE_INTERVAL :  0.01932111620903015
w_total_loss0.item()/UPDATE_INTERVAL :  0.019334126710891724
w_total_loss1.item()/UPDATE_INTERVAL :  0.019333648681640624
| epoch 142 |     0/  184 batches | ms/batch  4.86 | s_loss  0.02  0.02 | w_loss  0.02  0.02
keyTime |||||||||||||||||||||||||||||||
train_function_time :  97.3536536693573
KeyTime |||||||||||||||||||||||||||||||
-----------------------------------------------------------------------------------------
| end epoch 142 | valid loss  7.86  7.86 | lr 0.00020|
-----------------------------------------------------------------------------------------
keyTime |||||||||||||||||||||||||||||||
one_epoch_time :  123.34862661361694
KeyTime |||||||||||||||||||||||||||||||
s_total_loss0.item()/UPDATE_INTERVAL :  0.019314128160476684
s_total_loss1.item()/UPDATE_INTERVAL :  0.019314154386520385
w_total_loss0.item()/UPDATE_INTERVAL :  0.019324601888656617
w_total_loss1.item()/UPDATE_INTERVAL :  0.01932440757751465
| epoch 143 |     0/  184 batches | ms/batch  4.84 | s_loss  0.02  0.02 | w_loss  0.02  0.02